## Find the lowest common ancestor of two nodes in a binary tree

Find the paths to the two nodes from the root. The last common node in the two pathes is lca. For example, if one path is a-b-c and the other is a-b-d, b is the lca. Or the first intersecting element in the two paths from the two nodes to the root.

Note that the algorithms below don't consider duplicates in the input tree.

In [5]:
class Tree(object):
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None
        self.parent = None
        
    def __str__(self):
        return str(self.key)
    
    def set_left(self, tree):
        self.left = tree
        tree.parent = self

    def set_right(self, tree):
        self.right = tree
        tree.parent = self

def search_tree(tree, key):
    if not tree:
        return None
    if tree.key == key:
        return tree
    left = search_tree(tree.left, key)
    right = search_tree(tree.right, key)
    return left if left else right

In [6]:
a = Tree(1)
b = Tree(2)
c = Tree(3)
d = Tree(4)
e = Tree(5)
a.set_left(b)
a.set_right(c)
b.set_left(d)
b.set_right(e)
binary_tree = a
print(search_tree(a, 2))
print(search_tree(a, 3))
print(search_tree(a, 6))

2
3
None


In [88]:
def path_from_root(tree, path):
    if tree.parent:
        path_from_root(tree.parent, path)
    path.append(tree)
    
def print_path(path):
    print 'path:',
    for tree in path:
        print tree,
    print

In [89]:
def find_lca(tree, x, y):
    tree_x = search_tree(tree, x)
    path_x = []
    path_from_root(tree_x, path_x)
    tree_y = search_tree(tree, y)
    path_y = []
    path_from_root(tree_y, path_y)
    for tree1, tree2 in zip(path_x, path_y):
        if not tree1 == tree2:
            break
        lca = tree1
    return lca

In [90]:
lca = find_lca(a, 3, 5)
print('lca: {}'.format(lca))
lca = find_lca(a, 4, 5)
print('lca: {}'.format(lca))

lca: 1
lca: 2


In [102]:
# A more efficient, recursive approach, O(n)
def find_lca(tree, x, y):
    if not tree:
        return None
    
    if tree.key == x or tree.key == y:
        return tree
    
    left = find_lca(tree.left, x, y)
    right = find_lca(tree.right, x, y)
    
    if left and right:
        return tree
    
    return left if left else right

In [103]:
lca = find_lca(a, 3, 5)
print(lca)
lca = find_lca(a, 4, 5)
print(lca)

1
2


What if there are duplicates in a tree? There can be multiple common ancestors. The algorithm below may work.

In [25]:
found = False
lca = None
count = 0

def find_lca(tree, x, y, wanted):
    global found, lca, count
    if not found:
        for v in [tree.left, tree.right]:
            if v:
                find_lca(v, x, y, wanted)
        if count == 2 and not found:
            lca = tree
            found = True
            return
        if tree.key not in wanted and (tree.key == x or tree.key == y):
            count += 1
wanted = []
find_lca(a, 3, 5, wanted)
assert lca.key == 1

## Find the lowest common ancestor of two nodes in a binary search tree

The first algorithm for binary trees above works for binary search trees as well. There is a more efficient algorithm based on a property of binary search trees, the keys in the left subtree is always less than those of the right subtree. Recursively check the key of each node. The first node whose key is between x and y or equals to x or y is the lca.

In [2]:
def insert(tree, x, parent=None):
    if not tree:
        tree = Tree(x)
        tree.parent = parent
        if parent.key > x:
            parent.left = tree
        else:
            parent.right = tree
    else:
        if tree.key > x:
            insert(tree.left, x, tree)
        else:
            insert(tree.right, x, tree)
            
def inorder_traverse(tree):
    if tree:
        inorder_traverse(tree.left)
        process_tree(tree)
        inorder_traverse(tree.right)

def process_tree(tree):
    print(tree.key)

In [3]:
tree = Tree(4)
insert(tree, 2)
insert(tree, 1)
insert(tree, 3)
insert(tree, 5)
inorder_traverse(tree) # This is for validating if the tree is a BST.

1
2
3
4
5


In [7]:
def find_lca(tree, x, y):
    if not tree:
        return
    # The equal signs for cases where the lca is one of the two nodes.
    if min(x, y) <= tree.key <= max(x, y):
        return tree
    if tree.key > max(x, y):
        return find_lca(tree.left, x, y)
    elif tree.key < min(x, y):
        return find_lca(tree.right, x, y)

lca = find_lca(tree, 3, 5)
assert lca.key == 4
lca = find_lca(tree, 4, 5)
assert lca.key == 4

## Validate a binary search tree

Algorithm 1:
Do in-order traverse. The current node should be greater than the previously processed node.
O(n)

Algorithm 2: min-max
The root, a, must be in [-inf, inf].
The left child, b, of the root must be in [-inf, a].
The right child, c, of the root must be in [a, inf].
The left child, d, of b must be in [-inf, b].
The right child, e, of b must be in [b, a].
The left child, f, of c must be in [a, c].
The right child, g, of c must be in [c, inf]
...
O(n)

In [99]:
def process_tree(tree):
    global stack
    stack.append(tree.key)

def validate_bst(tree):
    global stack
    inorder_traverse(tree)
    prev_key = -float('inf')
    for key in stack:
        if key < prev_key:
            return False
        prev_key = key
    return True
stack = []
assert validate_bst(bst) is True
stack = []
assert validate_bst(binary_tree) is False

In [105]:
def validate_bst(tree, min_value, max_value):
    if not tree:
        return True
    if tree.key < min_value or tree.key > max_value:
        return False
    return validate_bst(tree.left, min_value, tree.key) and validate_bst(tree.right, tree.key, max_value)
min_value = -float('inf')
max_value = float('inf')
assert validate_bst(bst, min_value, max_value)
assert not validate_bst(binary_tree, min_value, max_value)

## Two Sum

O(n) using a hash table

Ask if there are duplicates in the array. For example, [1, 1, 1] and sum = 2 should result in (1, 1).

In [3]:
def find_twosum(target, data):
    dict_ = {}
    result = []
    for i, _ in enumerate(data):
        if data[i] in dict_:
            result.append((dict_[data[i]], i))
        else:
            dict_[target - data[i]] = i
    return result

data = [1, 7, 3, 8, 9]
target = 10
print find_twosum(target, data)

[(1, 2), (0, 4)]


## Three Sum

```
sort(s)
for i = 0 to n-3
    a = s[i]
    start = i + 1
    end = n - 1
    while start < end
        b = s[start]
        c = s[end]
        if a + b + c = 0 then
            output a, b, c
            end -= 1
        else if a + b + c > 0 then
            end -= 1
        else
            start += 1
```
Worst case O(n^2)

In [8]:
def find_threesum(target, data):
    data = sorted(data)
    result = []
    n = len(data)
    for i in range(n - 2):
        a = data[i]
        start = i + 1
        end = n - 1
        while start < end:
            b = data[start]
            c = data[end]
            if a + b + c == target:
                result.append((a, b, c))
                start += 1
                end -= 1
            elif a + b + c > target:
                end -= 1
            else:
                start += 1
    return result

data = [1, 8, 2, 3, 9, 7, 4, 5]
target = 10
print(find_threesum(target, data))

[(1, 2, 7), (1, 4, 5), (2, 3, 5)]


### Find two elements in an array, whose absolute difference is some number.

Could be solved similarily to the two sum. Or solve differently with a hash table in O(n).

[1, 1, 3, 4, 5]  k = 2
[1, 1, 1] k = 2
What if k = 0?

## Evaluate infix expressions

This is a infix-to-postfix conversion algorithm but the logic is same:

```
Scan the token list from left to right.
If the token is an operand, append it to the end of the output list.
If the token is a left parenthesis, push it on the opstack.
If the token is a right parenthesis, pop the opstack until the corresponding left parenthesis is removed. Append each operator to the end of the output list.
If the token is an operator, *, /, +, or -, push it on the opstack. However, first remove any operators already on the opstack that have higher or equal precedence and append them to the output list.
When the input expression has been completely processed, check the opstack. Any operators still on the stack can be removed and appended to the end of the output list.```

In [8]:
from tokenize import generate_tokens
from StringIO import StringIO

def validate_expr(expr):
    # If expr starts with -, pad with 0.
    # All parenteses are balanced.
    pass

def compute(num2, num1, op):
    print 'Compute {} {} {}'.format(num1, op, num2)
    if op == '+':
        return num1 + num2
    elif op == '-':
        return num1 - num2
    elif op == '*':
        return num1 * num2
    elif op == '/':
        return num1 / num2
    else:
        print('Invalid operator')
        return None
    
def eval_infix(expr):
    num_stack = []
    op_stack = []
    priority = {'(': 0, ')': 0, '+': 1, '-': 2, '*': 3, '/': 3}

    # Handle the leading negative symbol
    if expr.startswith('-'):
        # Could it be just inserting '0' in the head? 
        expr = '0' + expr
    
    tokens = generate_tokens(StringIO(expr).readline)
    for token_num, token_val, _, _, _ in tokens:
        if not token_val:
            break
        elif token_val.isdigit():
            num_stack.append(float(token_val))
        elif token_val == '(':
            op_stack.append(token_val)
        elif token_val == ')':
            while not op_stack[-1] == '(':
                num_stack.append(compute(num_stack.pop(), num_stack.pop(), op_stack.pop()))
            op_stack.pop()
        else: # token_val is an operator.
            while op_stack and priority[token_val] < priority[op_stack[-1]]:
                num_stack.append(compute(num_stack.pop(), num_stack.pop(), op_stack.pop()))
            op_stack.append(token_val)
    
    while op_stack:
        num_stack.append(compute(num_stack.pop(), num_stack.pop(), op_stack.pop()))
    
    return num_stack[0] if num_stack else 0

expr = '-12+2*(1-9)'
assert eval_infix(expr) == -28.0

Compute 0.0 - 12.0
Compute 1.0 - 9.0
Compute 2.0 * -8.0
Compute -12.0 + -16.0


## Max. No of Meeting Rooms

```
Flatten the (start, end) pairs.
Sort the times.
For each time
  If the time is start
    count++
  else
    count--
Return max. count
```

memory O(n)

time O(nlogn) for sorting the list

where n is the no. of meetings

In [20]:
meetings = [(1, 4), (5, 6), (8, 9), (2, 6)]
times = []
for time in meetings:
    times.append((time[0], 0))
    times.append((time[1], 1))

times.sort(key = lambda time: time[0])

count = 0
max_count = 0
for time, type_ in times:
    if type_ == 0:
        count += 1
    else:
        count -= 1
    max_count = max(count, max_count)

assert max_count == 2

## Check if a query word with a wildcard exists in a given list of words


Use trie. Search time complexity is O(n) where n is the length of the query. Be careful with some edge conditions.

https://stackoverflow.com/questions/11015320/how-to-create-a-trie-in-python


In [14]:
def init_tree(*words):
    trie = {}
    for word in words:
        d = trie
        for letter in word:
            d = d.setdefault(letter, {})
        d['end'] = True
    return trie

trie = init_tree('bar', 'bat', 'car', 'cat', 'acc', 'aab')
assert trie['b']['a']['r']['end'] == True
assert trie['b']['a']['t']['end'] == True
assert trie['c']['a']['r']['end'] == True
assert trie['c']['a']['t']['end'] == True

def search(trie, word):
    d = trie
    for letter in word:
        try:
            d = d[letter]
        except KeyError:
            return False
    return 'end' in d    
        
assert search(trie, 'bar') == True
assert search(trie, 'tar') == False
assert search(trie, 'ba') == False

def search_regex(trie, word):
    d = trie
    for i, letter in enumerate(word):
        if letter == '.': # Wildcard
            for _, v in d.iteritems():
                if search_regex(v, word[i+1:]):
                    return True
            return False
        elif letter == '*': # One or more of the previous character
            try:
                prev_letter = word[i-1]
                while prev_letter in d:
                    d = d[prev_letter]
            except IndexError:
                print 'Illegal argument {}'.format(word)
        else:
            try:
                d = d[letter]
            except KeyError:
                return False
    return 'end' in d

assert search_regex(trie, 'c.t') == True
assert search_regex(trie, 'c.s') == False
assert search_regex(trie, 'c..') == True
assert search_regex(trie, 'ac*') == True
assert search_regex(trie, 'a*c') == False
assert search_regex(trie, 'a*.') == True

Recursive implementation

In [7]:
def add_word(trie, word):
    if not word:
        return
    trie = trie.setdefault(word[0], {})
    if len(word) == 1:
        trie['end'] = True
    else:
        add_word(trie, word[1:])

def search(trie, word):
    if not word:
        return False
    try:
        trie = trie[word[0]]
    except KeyError:
        return False
    if len(word) == 1:
        return 'end' in trie
    else:
        return search(trie, word[1:])

def search_regex(trie, word, prev=None):
    if not trie:
        return False
    if len(word) == 1:
        if word[0] == '.':
            return any('end' in v for _, v in trie.iteritems())
        elif word[0] == '*':
            while prev in trie:
                trie = trie[prev]
            return 'end' in trie
        else:
            try:
                return 'end' in trie[word[0]]
            except KeyError:
                return False
    if word[0] == '.':
        return any(search_regex(v, word[1:]) for _, v in trie.iteritems())
    elif word[0] == '*':
        if not prev:
            print 'Invalid syntax'
            return False
        else:
            try:
                return search_regex(trie[prev], word[1:], prev)
            except KeyError:
                return False
    else:
        try:
            return search_regex(trie[word[0]], word[1:], word[0])
        except KeyError:
            return False

trie = {}
add_word(trie, 'bar')
add_word(trie, 'bat')
add_word(trie, 'car')
add_word(trie, 'cat')
add_word(trie, 'accc')
add_word(trie, 'aab')

assert search(trie, 'bat') == True
assert search(trie, 'bot') == False
assert search_regex(trie, 'c.t') == True
assert search_regex(trie, 'c.s') == False
assert search_regex(trie, 'c..') == True
assert search_regex(trie, 'ac*') == True
assert search_regex(trie, 'a*c') == False
assert search_regex(trie, 'a*.') == True

### Regular expression matching

Implement regular expression matching with support for '.' and '*'.

'.' Matches any single character.
'*' Matches zero or more of the preceding element.

The matching should cover the entire input string (not partial).

The function prototype should be:
bool isMatch(const char *s, const char *p)

Some examples:
isMatch("aa","a") → false
isMatch("aa","aa") → true
isMatch("aaa","aa") → false
isMatch("aa", "a*") → true
isMatch("aa", ".*") → true
isMatch("ab", ".*") → true
isMatch("aab", "c*a*b") → true

In [4]:
class Solution(object):
    def isMatch(self, s, p):
        """
        :type s: str
        :type p: str
        :rtype: bool
        """
        if not p:
            return len(s) == 0
        
        if len(p) == 1:
            return len(s) == 1 and (s[0] == p[0] or p[0] == '.')         
        
        if p[1] == '*':
            return (not len(s) == 0 and (s[0] == p[0] or p[0] == '.') and self.isMatch(s[1:], p)) or self.isMatch(s, p[2:])
        else:
            return not len(s) == 0 and (s[0] == p[0] or p[0] == '.') and self.isMatch(s[1:], p[1:])
s = "a"
p = ".*..a*"
sol = Solution()
sol.isMatch(s, p)

False

In [3]:
'' == True

False

## Find the sorted uninon of two sorted lists

Something similar to the merge part of the merge sort

In [19]:
a1 = [1, 3, 5, 7]
a2 = [2, 3, 4, 5, 6, 7]
n1 = len(a1)
n2 = len(a2)
result = []
i = j = 0
while i < n1 and j < n2:
    if a1[i] < a2[j]:
        result.append(a1[i])
        i += 1
    elif a1[i] > a2[j]:
        result.append(a2[j])
        j += 1
    else:
        result.append(a1[i])
        i += 1
        j += 1

while i < n1:
    result.append(a1[i])
    i += 1
    
while j < n2:
    result.append(a2[j])
    j += 1
    
assert result == [1, 2, 3, 4, 5, 6, 7]

What if the input lists are sorted linked lists? Find whose head is smaller. Merge the larger head to the smaller one.

In [5]:
class Node(object):
    def __init__(self, x):
        self.data = x
        self.next = None
    
    def __str__(self):
        return str(self.data)

def print_list(l):
    while l:
        print l
        l = l.next

def union_util(l1, l2):
    # Assume that l1 > l2.
    while l1 and l2:
        if l1.data <= l2.data:
            prev = l1
            l1 = l1.next
        else:
            prev.next = l2
            l2.next, l2 = l1, l2.next
            
    while l2:
        prev.next = l2
        l2 = l2.next
        prev = prev.next

def union(l1, l2):
    if l1.data <= l2.data:
        union_util(l1, l2)
        return l1
    else:
        union_util(l2, l1)
        return l2

l1 = Node(1)
l1.next = Node(2)
l1.next.next = Node(4)

l2 = Node(3)
l2.next = Node(5)

head = union(l1, l2)
print_list(head)

1
2
3
4
5


## Palindrome

In [9]:
def palindrome(x, low, high):
    if low >= high:
        return True
    return x[low] == x[high] and palindrome(x, low + 1, high - 1)

x = 'madam'
assert palindrome(x, 0, len(x) - 1) == True
x = 'maddam'
assert palindrome(x, 0, len(x) - 1) == True
x = 'loel'
assert palindrome(x, 0, len(x) - 1) == False
x = 'lotel'
assert palindrome(x, 0, len(x) - 1) == False

## Maximum Contiguous Sum

Iterate through the elements and calculate the contiguous sum. Once the sum becomes negative, it doesn't help increase the contiguous sum. It'd better start over. Return the max. of the contiguous sums.

In [4]:
def max_contiguous_sum(x):
    n = len(x)
    s = n*[0]
    s[0] = x[0]
    
    # The following for-loop can be replaced with the following one liner:
    # s[i] = max(s[i - 1] + x[i], x[i])
    for i in range(1, n):
        if s[i - 1] < 0:
            s[i] = x[i]
        else:
            s[i] = s[i - 1] + x[i]
    return max(s)

x = [1, 3, -1, 2, -6, 4]
assert max_contiguous_sum(x) == 5

Return the max sum subarray.

In [2]:
class Solution(object):
    def maxSubArray(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        n = len(nums)
        s = n*[0]
        s[0] = nums[0]
        start = 0
        end = 0
        max_sum = s[0]
        for i in range(1, n):
            if s[i-1] < 0:
                s[i] = nums[i]
                start = i
            else:
                s[i] = s[i-1] + nums[i]
                end = i
            if s[i] > max_sum:
                max_sum = s[i]
                max_start = start
                max_end = end
        return nums[max_start:max_end+1]

nums = [-2,1,-3,4,-1,2,1,-5,4]
sol = Solution()
sol.maxSubArray(nums)

[4, -1, 2, 1]

## Longest Increasing Subsequence

The longest increasing subsequence at position i is the longest increasing subsequnce so far at position j added by 1 only if x[i] > x[j].


In [3]:
def longest_increasing_subsequence(x):
    n = len(x)
    l = n*[0]
    l[0] = 1
    for i in range(1, n):
        max_length = 0
        for j in range(i):
            if x[i] > x[j]:
                max_length = max(max_length, l[j])
        l[i] = max_length + 1
    return max(l)

x = [1, 3, -1, 2, -6, 4]
assert longest_increasing_subsequence(x) == 3

## BFS

From the search tree, one can find the shortest path to each vertex from the starting vertex. Note that find_path(u, v, parent) is valid only when bfs is done starting from u. 

In [2]:
from collections import defaultdict

adj = {}
adj[1] = [2, 5, 6]
adj[2] = [1, 3, 5]
adj[3] = [2, 4]
adj[4] = [3, 5]
adj[5] = [1, 2, 4]
adj[6] = [1]

parent = defaultdict(lambda: None)
def init_search():
    for key in parent.keys():
        parent[key] = None

def bfs(adj, start):
    init_search()
    discovered = defaultdict(lambda: False)
    processed = defaultdict(lambda: False)
    queue = []
    
    discovered[start] = True
    queue.append(start)
    while queue:
        u = queue.pop(0)
        for v in adj[u]:
            if not discovered[v]:
                discovered[v] = True
                parent[v] = u
                queue.append(v)
        process_vertex(u)
        processed[u] = True

def process_vertex(v):
    print v

def find_path(start, end, parent):
    stack = []
    v = end
    while v:
        stack.append(v)
        v = parent[v]
    
    print 'Path from {} to {}:'.format(start, end)
    while stack:
        print stack.pop()

print 'bfs starting from 1'
bfs(adj, 1)
find_path(1, 4, parent)

print 'bfs starting from 2'
bfs(adj, 2)
find_path(2, 5, parent)

bfs starting from 1
1
2
5
6
3
4
Path from 1 to 4:
1
5
4
bfs starting from 2
2
1
3
5
6
4
Path from 2 to 5:
2
5


In [8]:
parents = 5*[None]
parents[2] = 1
parents[3] = 1
parents[4] = 2

def find_path(start, end, parents):
    if start == end:
        return True
    if not end:
        return False
    return find_path(start, parents[end], parents)

assert find_path(1, 4, parents) == True
assert find_path(3, 4, parents) == False

## DFS

BFS with a stack instead of a queue. This is same as the recursive implmentation with printing each vertex in process_vertex_early(). This implementation cannot be used for topological sorting. In topological sorting each vertex should be pushed to a stach in process_vertex late().

In [3]:
from collections import defaultdict

adj = {}
adj[1] = [2, 5, 6]
adj[2] = [1, 3, 5]
adj[3] = [2, 4]
adj[4] = [3, 5]
adj[5] = [1, 4]
adj[6] = [1]

def dfs(adj, start):
    discovered = defaultdict(lambda: False)
    processed = defaultdict(lambda: False)
    stack = []
    
    discovered[start] = True
    stack.append(start)
    while stack:
        u = stack.pop()
        for v in adj[u]:
            if not discovered[v]:
                discovered[v] = True
                stack.append(v)
        process_vertex(u)
        processed[u] = True

def process_vertex(v):
    print v

dfs(adj, 1)

1
6
5
4
3
2


This dfs algorithm works for topological sorting. All vertices are initially white. A discovered vertex is grey while A process one is black.
1. color all vertices white
2. initialize an empty stack S
3. while there is still a white vertex u
4. color[u] = grey
5. vactive = u
6. do
7. if vactive has a white neighbor v
8. color[v] = grey
9. insert vactive into S
10. vactive = v
11. else
12. color[vactive ] = black
13. pop the top vertex of S, and set it to vactive
14. while vactive 6= ∅

## Find a postion to insert a number to a sorted array using binary search

In [27]:
def find_position(x, low, high, key):
    if high - low == 1:
        print 'high - low = 1. low {}, high {}'.format(low, high)
        if key < x[low]:
            return low
        elif key > x[high]:
            return high + 1
        else:
            return high

    if low == high:
        if x[low] > key:
            return low
        else:
            return low + 1

    if low > high:
        return

    mid = (low + high)/2
    if x[mid] == key:
        return mid
    if x[mid] > key:
        return find_position(x, low, mid - 1, key)
    else:
        return find_position(x, mid + 1, high, key)
        
x = [1, 3, 4, 8, 12]
n = len(x)
print find_position(x, 0, n - 1, 2)
print find_position(x, 0, n - 1, 3)
print find_position(x, 0, n - 1, 7)
print find_position(x, 0, n - 1, 9)
print find_position(x, 0, n - 1, 13)

high - low = 1. low 0, high 1
1
high - low = 1. low 0, high 1
1
high - low = 1. low 3, high 4
3
high - low = 1. low 3, high 4
4
high - low = 1. low 3, high 4
5


## Median of a stream of integers

- Unsorted array

O(1) for insert, O(n) for median

- Sorted array

median = x[n/2]

O(n) for insert, O(1) for median

- Binary search tree

In-order traverse. Return the n/2-th node.

O(h) for insert, O(n) for median. h is n in the worst case.

- Balanced binary search tree

Keep the no. of nodes in the left and right trees balanced. Specificaly, keep the left tree height equal or taller than the right tree by 1. The median is the root if the no. of nodes is even. Otherwise, the median is the avg. of the root and the logical predecessor of the root, i.e., the right most node of the left subtree of the root.

O(h) for insert, O(h) for median. h is n the worst case.
Finding median can be O(1) if update the pointer to the predecessor on each insertion.

- Two binary heaps

Store an incoming integer to either maxheap or minheap. Similarly to the balanced binary search tree, keep the size of maxheap equal to or greater than the size of minheap by 1. The median is the root of the maxheap when the no. of the integers odd. Otherwise, the avg. of the roots of the two heaps.

O(log n) for insertion, O(1) for median.

In [28]:
import heapq

def insert(x, left, right):
    global n_left, n_right
    if not left or x <= -left[0]:
        print 'push', x, 'to left'
        heapq.heappush(left, -x)
        n_left += 1
        if n_left - n_right > 1:
            print 'move', -left[0], 'to right'
            heapq.heappush(right, -heapq.heappop(left))
            n_left -= 1
            n_right += 1
    else:
        print 'push', x, 'to right'
        heapq.heappush(right, x)
        n_right += 1
        if n_left < n_right:
            print 'move', right[0], 'to right'
            heapq.heappush(left, -heapq.heappop(right))
            n_left += 1
            n_right -= 1

def median(left, right):
    return -left[0] if n_left > n_right else 0.5*(-left[0]+right[0])

integers = [1, 3, 5, 9, 7, 5]
left = []
right = []
n_left = 0
n_right = 0

for item in integers:
    insert(item, left, right)
    print median(left, right)

push 1 to left
1
push 3 to right
2.0
push 5 to right
move 3 to right
3
push 9 to right
4.0
push 7 to right
move 5 to right
5
push 5 to left
move 5 to right
5.0


## Write atoi()

Iterate from the first character to the end and calculate a running sum. At each digit, (running sum) * 10 + toInt(current character). If the string starts with '-', remember that. The iteration above should start from the frist numerical character. Return the running sum appropriately depending on the starting character. 

In [7]:
def atoi(string):
    chToNum = {"0": 0, "1": 1, "2": 2, "3": 3, "4": 4, "5": 5, "6": 6, "7": 7, "8": 8, "9": 9}
    isPositive = True
    if string.startswith('-'):
        isPositive = False
        string = string[1:]

    current_sum = 0
    for ch in string:
        if ch not in chToNum:
            print "Not a digig."
            return None
        current_sum = current_sum*10 + chToNum[ch]
    return current_sum if isPositive else -current_sum

atoi('-42')

-42

# Maksim Noy's List of Problems

## Linked List

In [1]:
class Node(object):
    def __init__(self, x):
        self.data = x
        self.next = None
    
    def __str__(self):
        return str(self.data)
    
def traverse_linked_list(l):
    result = []
    while l:
        result.append(l.data)
        l = l.next
    return result

### Unsorted List

In [2]:
# Replace the head with the new node
def insert(l, x):
    node = Node(x)
    node.next = l
    return node

myList = insert(None, 1)
myList = insert(myList, 2)
myList = insert(myList, 3)
assert traverse_linked_list(myList) == [3, 2, 1]

In [27]:
def search(l, x):
    if not l:
        return None
    if l.data == x:
        return l
    return search(l.next, x)

myList = Node(1)
myList = insert(myList, 2)
myList = insert(myList, 3)
assert search(myList, 1).data == 1
assert search(myList, 2).data == 2
assert search(myList, 3).data == 3
assert search(myList, 4) is None

In finding a predecessor the key is that you iteratively check node.next instead of node.

In [33]:
def predecessor(l, x):
    # null list or reach the end of the list
    if not l or not l.next:
        return None

    if l.data == x: # if head contains x.
        return None

    while l.next:
        if l.next.data == x:
            return l
        l = l.next
    return None

# Recursive implentation
def predecessor(l, x):
    if not l or not l.next:
        return None
    if l.data == x: # if the head contains x
        return None
    if l.next.data == x:
        return l
    else:
        return predecessor(l.next, x)

myList = Node(1)
myList = insert(myList, 2)
myList = insert(myList, 3)
assert traverse_linked_list(myList) == [3, 2, 1]
assert predecessor(myList, 2).data == 3
assert predecessor(myList, 1).data == 2
assert predecessor(myList, 3) is None

### Delete all nodes that contain x.

In [4]:
def delete_all(l, x):
    head = l
    prev = None
    while l:
        if l.data == x:
            if prev:
                prev.next = l.next
            else:
                head = l.next
        prev = l
        l = l.next
    return head

myList = Node(1)
myList = insert(myList, 2)
myList = insert(myList, 3)
myList = insert(myList, 2)
assert traverse_linked_list(myList) == [2, 3, 2, 1]
myList = delete_all(myList, 2)
assert traverse_linked_list(myList) == [3, 1]

### Delete the first node that contain x
delete_all with early termination.

In [5]:
def delete_first(l, x):
    head = l
    prev = None
    while l:
        if l.data == x:
            if prev:
                prev.next = l.next
            else:
                head = l.next
            return head
        prev = l
        l = l.next
    return head

myList = Node(1)
myList = insert(myList, 2)
myList = insert(myList, 3)
myList = delete_first(myList, 2)
assert traverse_linked_list(myList) == [3, 1]

In [14]:
def split(l, pivot):
    lt, gt = None, None
    while l:
        if l.data < pivot:
            lt = insert(lt, l.data)
        else:
            gt = insert(gt, l.data)
        l = l.next
        
    return lt, gt

my_list = insert(None, 3)
my_list = insert(my_list, 5)
my_list = insert(my_list, 2)
my_list = insert(my_list, 1)
my_list = insert(my_list, 4)
assert traverse_linked_list(my_list) == [4, 1, 2, 5, 3]

lt, gt = split(my_list, 3)
assert traverse_linked_list(lt) == [2, 1]
assert traverse_linked_list(gt) == [3, 5, 4]

In [21]:
# This split implementation is more memory-efficient than the previous one because it re-wires nodes
# instead of creating a copy of each node. 

def insert_node(l, node):
    node.next = l
    return node

def split(l, pivot):
    lt, gt = None, None
    head = l
    while l:
        next_ = l.next
        if l.data < pivot:
            lt = insert_node(lt, l)
        else:
            gt = insert_node(gt, l)
        l = next_
    return lt, gt

my_list = insert(None, 3)
my_list = insert(my_list, 5)
my_list = insert(my_list, 2)
my_list = insert(my_list, 1)
my_list = insert(my_list, 4)
assert traverse_linked_list(my_list) == [4, 1, 2, 5, 3]

lt, gt = split(my_list, 3)
assert traverse_linked_list(lt) == [2, 1]
assert traverse_linked_list(gt) == [3, 5, 4]

### Sorted List

Think about the following cases:
- The given list is empty.
- The head of the given list is greather than the inserting value.
- The inserting value is between the head and the tail.

In [19]:
# Insert to a sorted list
def insert_sorted(l, x):
    new_node = Node(x)
    
    # If the list is empty...
    if not l:
        return new_node
    
    head = l
    # If head is greather than x...
    if l.data >= x:
        new_node.next = l
        head = new_node
        return head

    while l.next:
        if l.next.data < x:
            l = l.next
        else:
            l.next, new_node.next = new_node, l.next
            return head
    
    # If all nodes in the list are less than x...
    if not l.next:
        l.next = new_node
        return head

myList = Node(2)
myList = insert_sorted(myList, 3)
myList = insert_sorted(myList, 1)
assert traverse_linked_list(myList) == [1, 2, 3]

### Cycle in a Linked List
A. Use a hash table. Traverse the list. At each node check if its item is already in the hash table. If yes, the list has a cycle. Otherwise, store the item in the table.
B. Traverse with one-node forward and two-node forward. If two traversing reaches to the same node, there is a cycle in the list.

In [19]:
def find_cycle(l):
    l1 = l
    l2 = l
    while l1 and l2:
        l1 = l1.next        
        if l2.next:
            l2 = l2.next.next
        else:
            break
        if id(l1) == id(l2):
            return True
    return False

my_list = insert(None, 1)
my_list = insert(my_list, 2)
my_list = insert(my_list, 3)
my_list = insert(my_list, 4)
my_list = insert(my_list, 5)
node1 = search(my_list, 1)
node2 = search(my_list, 2)
node5 = search(my_list, 5)
node1.next = node5
assert find_cycle(my_list) == True

### Middle of a Linked List
Traverse the list in two ways: one-step forward and two-step forward. When the two-step forward traversing meets the end of the list, the node where the one-step forward traversing is located is the middle of the list.

In [4]:
def find_middle(l):
    l1 = l
    l2 = l
    while l2:
        if l2.next:
            l2 = l2.next.next
            l1 = l1.next
        else:
            break
    return l1.data

my_list = insert(None, 1)
my_list = insert(my_list, 2)
my_list = insert(my_list, 3)
my_list = insert(my_list, 4)
my_list = insert(my_list, 5)
assert find_middle(my_list) == 3

In [ ]:
### Intersection of two linked lists

Write a program to find the node at which the intersection of two singly linked lists begins.

For example, the following two linked lists:

A:          a1 → a2
                   ↘
                     c1 → c2 → c3
                   ↗            
B:     b1 → b2 → b3
begin to intersect at node c1.

# the idea is if you switch head, the possible difference between length would be countered. 
# On the second traversal, they either hit or miss. 
# if they meet, pa or pb would be the node we are looking for, 
# if they didn't meet, they will hit the end at the same iteration, pa == pb == None, return either one of them is the same,None

In [ ]:
def getIntersectionNode(self, headA, headB):
    """
    :type head1, head1: ListNode
    :rtype: ListNode
    """
    l1 = headA
    l2 = headB
    while l1 is not l2:
        if l1:
            l1 = l1.next
        else:
            l1 = headB
        if l2:
            l2 = l2.next
        else:
            l2 = headA
    return l1

### Reverse the Individual Words in a String 

In [16]:
def reverse_words(s):
    s = list(s)
    found_start = False
    start = 0
    end = 0
    for i in range(len(s)):
        if not found_start:
            if s[i] == ' ':
                continue
            else:
                start = i
                found_start = True
        else:
            if s[i] == ' ':
                end = i - 1
                reverse(s, start, end)
                found_start = False
            else:
                continue
    s = ''.join(s)
    return s
    
def reverse(l, low, high):
    while low < high:
        l[low], l[high] = l[high], l[low]
        low += 1
        high -= 1

l = list(' hello ')
reverse(l, 1, 5)
s = ''.join(l)
assert s == ' olleh '

s = ' hello  world '
assert reverse_words(s) == ' olleh  dlrow '

### Reverse the order of the words in a String
First, reverse the whole string. Second, reverse each word.

Clarify how to handle spaces.

In [17]:
def reverse_words_order(s):
    s = ''.join(reversed(s))
    return reverse_words(s)

s = ' hello world  '
assert reverse_words_order(s) == '  world hello '

### Strip Whitespaces in a String
The initial value for the index of the first space is 0. Iterate over the characters. Swap if the character of the current position is not a space. Move the position of the first space to the right. Similar to Skiena's implementation of quicksort. In the end strip off all trailing spaces. O(n)

In [2]:
def strip_spaces(s):
    first_space = 0
    s = list(s)
    for i, char in enumerate(s):
        if not char == ' ':
            swap(s, first_space, i)
            first_space += 1
    s = s[:first_space]
    return ''.join(s)

def swap(l, i, j):
    l[i], l[j] = l[j], l[i]

s = ' hello  world '
assert strip_spaces(s) == 'helloworld'

### Remove duplicate characters in a sorted string
'aa  bbb' -> 'a b'

Similar to the strip whitespaces problem above. Variation of quickseletion. O(n) time.

In [3]:
def dedup(s):
    s = list(s)
    j = 0
    for i in range(1, len(s)):
        if not s[i] == s[j]:
            j += 1
            s[j] = s[i]
    return ''.join(s[:j+1])

s = 'aa bbb'
dedup(s)

'a b'

A variation of the problem above. Some integers in an array are duplicate in a sorted manner. Dedup the array.

122333 -> 123

In [ ]:
class Solution(object):
    def removeDuplicates(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        if not nums:
            return 0
        n = len(nums)
        if n == 1:
            return 1
        head = 1
        tail = 0
        while head < n:
            if not nums[head] == nums[tail]:
                tail += 1
                nums[tail] = nums[head]
            head += 1
        return tail + 1

### Find the first non-repeating character in a string
'abca' -> 'b'

A.

Store the count of each alphabet in a dictionary.

Scan the string. For each char, increment its count.

Scan again. For each char, return if its count is one.

O(len(s)), but have to scan twice.

B.

If the string is long and the size of alphabets is small, algorithm A is inefficient.

Store the (count, the index of the first appearance) tuples in a dictionary.

Scan the string. For each char, increment its count.

Scan the count dictionary instead of the string. Return the char at the min index among the chars whose counts are one.

Still O(len(s)) if len(s) > len(alphabets), but the second scanning is much faster than the first scanning.

In [13]:
from collections import defaultdict

def unique_char(s):
    counts = defaultdict(lambda: (0, 0))
    for i, char in enumerate(s):
        if counts[char][0] == 0:
            counts[char] = (1, i)
        else:
            count = counts[char][0] + 1
            index = counts[char][1]
            counts[char] = (count, index)
        
    min_index = len(s)
    for k, (count, index) in counts.iteritems():
        if count == 1:
            min_index = min(min_index, index)
    
    return s[min_index]
    
assert unique_char('abca') == 'b'

### Longest increasing subsequence
A. Dynamic programming, O(n^2)

In [9]:
def longest_increasing_subsequence(s):
    n = len(s)
    l = n*[0]
    l[0] = 1
    for i in range(n):
        for j in range(i):
            if s[i] > s[j]:                    
                l[i] = max(l[i], l[j] + 1)
    return max(l)

assert longest_increasing_subsequence([1, 3, 2, 7, 8, 5]) == 4

Can you return the longest increasing subsequnce, too?

In [19]:
def longest_increasing_subsequence(s):
    n = len(s)
    l = n*[0]
    l[0] = 1
    subseq = [[] for i in range(n)]
    subseq[0] = [s[0]]

    for i in range(n):
        for j in range(i):
            if s[i] > s[j]:
                length = l[j] + 1
                if l[i] < length:
                    l[i] = length
                    subseq[i] = subseq[j] + [s[i]] 
    
    max_length = max(l)
    longest_subseq = subseq[l.index(max_length)] # index returns the first occurence.
    return max_length, longest_subseq

length, subseq = longest_increasing_subsequence([1, 3, 2, 7, 8, 5])
assert length == 4
assert subseq == [1, 3, 7, 8]

Topological sorting and longest path from each node in DAG. O(n + m) but m ~ O(n^2) so that no efficiency gain.

### Max contiguous sum
Scan the array and calculate the contiguous sum up to the current position. If adding the current item to the contiguous sum up so far doesn't increase the contiguous sum, reset. That is when the contiguous sum becomes negative.

In [9]:
def max_countiguous_sum(a):
    n = len(a)
    current_sum = a[0]
    max_sum = 0
    for i in range(1, n):
        current_sum = max(a[i], current_sum + a[i])
        max_sum = max(max_sum, current_sum)
    return max_sum

assert max_countiguous_sum([1, 3, -5, 1, 2]) == 4

Return the contiguous elements that results in the max sum as well.

In [10]:
def max_contiguous_sum(a):
    n = len(a)
    s = n*[0]
    current_sum = a[0]
    start = 0
    end = 0
    max_sum = 0
    for i in range(1, n):
        if s[i - 1] < 0:
            current_sum = a[i]
            start = i
        else:
            current_sum = current_sum + a[i]
            end = i
        if current_sum > max_sum:
            max_sum = current_sum
            max_start = start
            max_end = end
    return max_sum, max_start, max_end

assert max_contiguous_sum([1, 3, -5, 1, 2]) == (4, 0, 1)

### Binary Search Tree
Base class and helper methods

In [5]:
class Tree(object):
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        self.parent = None

    def __str__(self):
        return str(self.data)

def in_order_print(tree):
    if tree:
        in_order_print(tree.left)
        print tree
        in_order_print(tree.right)

Since it's hard to manipulate pointers in python, we have to do differently from c implementations.

In [6]:
def insert(tree, data):
    if tree:
        if tree.data > data:
            if not tree.left:
                new_tree = Tree(data)
                new_tree.parent = tree
                tree.left = new_tree
            else:
                insert(tree.left, data)
        else:
            if not tree.right:
                new_tree = Tree(data)
                new_tree.parent = tree
                tree.right = new_tree
            else:
                insert(tree.right, data)

tree = Tree(3)
insert(tree, 7)
insert(tree, 1)
insert(tree, 5)
in_order_print(tree)

1
3
5
7


Another implementation. Pass parent to the insert method.

In [4]:
def insert(tree, data, parent=None):
    if not tree:
        new_tree = Tree(data)
        new_tree.parent = parent
        if parent:
            if parent.data > data:
                parent.left = new_tree
            else:
                parent.right = new_tree
    else:
        if tree.data > data:
            insert(tree.left, data, tree)
        else:
            insert(tree.right, data, tree)
            
tree = Tree(3)
insert(tree, 7)
insert(tree, 1)
insert(tree, 5)
in_order_print(tree)

1
3
5
7


In [19]:
import sys

def check_bst(tree, lb, ub):
    if tree:
        return (lb < tree.data < ub and
                check_bst(tree.left, lb, tree.data) and
                check_bst(tree.right, tree.data, ub))

    else:
        return True

tree = Tree(3)
insert(tree, 7)
insert(tree, 1)
insert(tree, 5)

lb = -sys.maxint - 1
ub = sys.maxint
assert check_bst(tree, lb, ub) == True

tree = Tree(3)
tree.left = Tree(7)
insert(tree, 1)
insert(tree, 5)

lb = -sys.maxint - 1
ub = sys.maxint
assert check_bst(tree, lb, ub) == False

### Find an integer that appear twice in an array of integers ranging from 1 to 100,000.
A. Linear search. Store the count of each integer in a hash table. O(n), two passes, extra memory for the hash table.

B. Sort. If an integer is the same integer at the previous position, it's the duplicate. O(nlogn). Maybe extra memory in sorting.

Clarify if beside of the duplicate integer all other integers appear in the array.

C. Sum all integers. Substract integers from 1 to 100,000, once for each. O(n), basically two pases. No extra memory.

D. If all integers appear once, ideal_sum = n(n+1)/2. Substract this from the sum of all integers. O(n), but one pass.

### Find an missing integer from an array of integers rainging from 1 to 100,000.

A. Hash table. O(n)

B. Sort. O(nlogn)

C. Sum all and substract 1 to 100,000. O(n)

D. Sum all and substract it from n(n+1)/2. O(n)

E. XOR trick. XOR all numbers in the array, i.e., xor ^= a[i] for i in a. XOR from 1 to 100,000, xor ^= i for i from 1 to 100,000.

### Set Mismatch
A set of interges ranging from 1 to n where n is the size of the set. One item is duplicated, e.g., [1, 2, 3, 4] >> [1, 2, 2, 4]. What are the two numbers, one duplicated and the other overwritten.

Use the set as a hash table.

In [ ]:
def findErrorNums(nums):
    for k in nums:
        if nums[abs(k)-1] > 0:
            nums[abs(k)-1] *= -1
        else:
            dup = abs(k)

    for i, k in enumerate(nums):
        if k > 0:
            miss = i + 1

    return [dup, miss]

### Shuffle a deck of cards

A. Random Swapping
```
Put the 52 cards in an array.
itermax = a large number
while iter < itermax:
    Generate two random numbers between 0 and 51.
    swap(cards, r1, r2)
    iter++
```

B. Fisher-Yates Shuffle
```
Put the 52 cards in an array.
m = 51
while m > 0:
    Generate a random number from 0 to m.
    swap(cards, r, m)
    m--
```

### Sum two largest integers in an array

A. Linear search while maintianing two largest. O(n)

B. Sort. The sum of the last two. O(nlogn)

### Sum of n largest elements in an array of integers ranging from 0 to 9
A. Sort, O(nlogn)

B. Heap, O(n) for heap creating + O(klogn) for min

C. Linear search, O(n), at most 9 passes

In [22]:
def sum_largest_k(a, k):
    n = len(a)
    s = 0
    for target in range(9, 1, -1):
        i = 0
        while k > 0 and i < n:
            if a[i] == target:
                k -= 1
                s += target
            i += 1
    return s

assert sum_largest_k([1, 9, 3, 7, 5], 3) == 21

# Some Random Questions from Glassdoor

### Implement x to the power of y.

In [20]:
def pow(x, y):
    if y == 0:
        return 1
    return x*pow(x, y - 1)

assert pow(2, 3) == 8

### Find the number of ways to score S where possible points are 3, 5, and 10.

Variations include:

- A number of ways to climb a flight of N steps with 1, 2, or 3 steps at a time.

- A number of ways to give changes.

In [2]:
def ways_to_score(score):
    points = [3, 5, 10]
    N = (score + 1)*[0]
    N[0] = 1
    for s in range(1, score + 1):
        for p in points:
            if s >= p:
                N[s] += N[s - p]
                
    return N[score]

assert ways_to_score(8) == 2

### Rotate an array: Move the r number of elements at the tail to the head
Similar to reversing the order of words in text. Reverse the whole thing, then reverse the left part and the right part individually.

In [19]:
def reverse(a, low, high):
    while low < high:
        swap(a, low, high)
        low += 1
        high -= 1

def swap(a, i, j):
    a[i], a[j] = a[j], a[i]

def rotate(a, r):
    n = len(a)
    reverse(a, 0, n - 1)
    reverse(a, 0, r - 1)
    reverse(a, r, n - 1)
    
a = [1, 2, 3, 4, 5]
rotate(a, 2)
assert a == [4, 5, 1, 2, 3]

### Add two strings of integers.

In [26]:
def add(s1, s2):
    try:
        s1 = int(s1)
        s2 = int(s2)
        return s1 + s2
    except ValueError as e:
        print e
        return None
    
assert add('1', '2') == 3

### Height of a binary tree.

Ask if the tree is balanced. If so, h = logn

In [7]:
def height(tree):
    if not tree:
        return -1
    return max(height(tree.left), height(tree.right)) + 1

tree = Tree(3)
insert(tree, 7)
insert(tree, 1)
insert(tree, 5)

assert height(tree) == 2

Another implementation.

In [19]:
def find_height(tree, height):
    if not tree:
        return height - 1
    if not tree.left and not tree.right:
        return height
    return max(find_height(tree.left, height+1), find_height(tree.right, height+1))

assert find_height(tree, 0) == 2

### Diameter, i.e., the longest path, of a binary tree

In [14]:
# This won't work if...
# Linear tree
# A tree with single node
# LCA of the two ends of the longest path is not the root.

def diameter(tree):
    if not tree:
        print 'The input is a null tree.'
    return height(tree.left) + height(tree.right) + 2

tree = Tree(3)
insert(tree, 7)
insert(tree, 1)
insert(tree, 5)

assert diameter(tree) == 3

This will work for single node trees, linear trees, etc.

In [ ]:
class Solution(object):
    def diameterOfBinaryTree(self, root):
        """
        :type root: TreeNode
        :rtype: int
        """
        # graph with single node
        # linear graph
        # The root is not the lca of the two ends of the longest two paths from the root.
        if not root:
            return 0
        self.max_diameter = 0
        def height(tree):
            if not tree:
                return -1
            count = 0
            h1 = height(tree.left)
            h2 = height(tree.right)
            self.max_diameter =  max(self.max_diameter, h1 + h2 + 2)
            return max(h1, h2) + 1
        height(root)
        return self.max_diameter

### Height of a tree

In [6]:
adj = {}
adj[1] = [2, 5, 6]
adj[2] = [1, 3, 5]
adj[3] = [2, 4]
adj[4] = [3, 5]
adj[5] = [1, 2, 4]
adj[6] = [1]

discovered = (len(adj)+1)*[False]

def dfs(adj, start):
    max_height = -1
    discovered[start] = True
    for v in adj[start]:
        if not discovered[v]:
            height = dfs(adj, v)
            max_height = max(max_height, height)
    return max_height + 1

assert dfs(adj, 1) == 4

4

### Longest path in a undirected, unweighted graph
longest path = sum of the heights of the two tallest subtrees + 2 for connecting these two subtrees to the root

            root
         /   |   \
 subtree  subtree subtree ...

longest path = sum of the heights of the two tallest subtrees
             + 2 for connecting these two subtrees to the root

dfs, O(n + m)

In [15]:
adj = {}
adj[1] = [2, 5, 6]
adj[2] = [1, 3, 5]
adj[3] = [2, 4]
adj[4] = [3, 5]
adj[5] = [1, 2, 4]
adj[6] = [1]

discovered = (len(adj)+1)*[False]
max_path_len = 0

def dfs(adj, start):
    global max_path_len
    first_max = -1 # -1 here makes the height of a leaf zero.
    second_max = -1
    discovered[start] = True
    for v in adj[start]:
        if not discovered[v]:
            height = dfs(adj, v)
            # Improve: Instead of this if condition, keep track of max_path_len
            if height > first_max:
                first_max, second_max = height, first_max
            elif height > second_max:
                second_max = height

    max_path_len = first_max + second_max + 2 
    return first_max + 1

dfs(adj, 1)
assert max_path_len == 5

### Continuous elements in an array of integers

```
for i = 0, n-1
  block = [a[i]]
  for j = i+1, n-1
    insert a[j] to the block in sorted manner. 
    check if delta between each contigous two elements equals to 1.
    if so,
      length = j-i+1
      max_length = max(max_length, length)
return max_length
```
Use insertion sort. O(n^3).

Any duplicates in the array? If not, O(n^2) is possible. See below.

In [12]:
def continuous_elements(a):
    max_length = 0
    n = len(a)
    for i in range(n):
        high, low = a[i], a[i]
        length = 1
        for j in range(i + 1, n):
            high = max(high, a[j])
            low = min(low, a[j])
            if not high - low > j - i:
                length = j - i + 1
                max_length = max(max_length, length)
    return max_length

assert contigous_elements([3, 2, 4]) == 3
assert contigous_elements([3, 2, 5, 7, 6]) == 3

In [ ]:
# Can it be down with the sliding window approach?

### Create a binary search tree from an array
For a binary tree, the element in the middle will be the root. Build a tree with each of the left side and the right side, respectively, and assign to the left and right children of the root recursively. If the tree has to be a bst, sort the array first.

In [1]:
class Tree(object):
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        self.parent = None

def inorder(tree, nodes):
    if tree:
        inorder(tree.left, nodes)
        nodes.append(tree.data)
        inorder(tree.right, nodes)

def insert(tree, l):
    n = len(l)
    i = 0
    while i < n:
        if i + 1 < n:
            tree.right = Tree(l[i+1])
            tree.right.parent = tree
        if i + 2 < n:
            parent = Tree(l[i+2])
            tree.parent = parent
            parent.left = tree
        if tree.parent:
            tree = tree.parent
        i += 2
    return tree

l = [1, 3, 5, 6, 9]

tree = Tree(l[0])
tree = insert(tree, l)

nodes = []
inorder(tree, nodes)

assert nodes == l

In [3]:
# Recursive solution, O(n)
def convert(a):
    if not a:
        return
    mid = len(a)//2
    node = Tree(a[mid])
    node.left = convert(a[:mid])
    node.right = convert(a[mid+1:])
    return node

a = [1, 3, 5, 6, 9]
tree = convert(a)
nodes = []
inorder(tree, nodes)

assert nodes == a

### Copy a linked list
Similarily to the previous problem, use the pre-order traversal. Recursion is handy, i.e., do something for the current item and call the function recursively for the next item.

In [4]:
class Node(object):
    def __init__(self, data):
        self.data = data
        self.next = None

def print_list(l):
    if l:
        print l.data
        print_list(l.next)

def copy_util(input_, output):
    if not input_.next:
        return
    output.next = Node(input_.next.data)
    copy_util(input_.next, output.next)
    
def copy(input_):
    if input_:
        output = Node(input_.data)
        copy_util(input_, output)
        return output
        
input_ = Node(1)
input_.next = Node(2)
input_.next.next = Node(3)

list_ = copy(input_)
print_list(list_)

1
2
3


### Find all subsets of a set, i.e., power set

In [14]:
def find_combinations(numbers, select, index, combinations):
    if index == len(numbers) - 1:
        select[index] = False
        combinations.add(frozenset([numbers[i] for i in range(len(numbers)) if select[i]]))
        select[index] = True
        combinations.add(frozenset([numbers[i] for i in range(len(numbers)) if select[i]]))
        return

    select[index] = False
    find_combinations(numbers, select, index + 1, combinations)
    select[index] = True
    find_combinations(numbers, select, index + 1, combinations)

numbers = [1, 2]
select = len(numbers)*[False]
index = 0
combinations = set()
find_combinations(numbers, select, index, combinations)
assert combinations == set([frozenset([]), frozenset([1]), frozenset([2]), frozenset([1, 2])])

### Cartesian Product
Given a digit string, return all possible letter combinations that the number could represent.

A mapping of digit to letters (just like on the telephone buttons) is given below.

In [ ]:
class Solution(object):
    def letterCombinations(self, digits):
        """
        :type digits: str
        :rtype: List[str]
        """
        letters = {'1':'', '2':'abc', '3':'def', '4':'ghi', '5':'jkl', '6':'mno', '7':'pqrs', '8':'tuv', '9':'wxyz', '0':' '}
        if not digits:
            return []
        return cartesian(digits, letters, 0)
            
        
def cartesian(nums, letters, index):
    if index == len(nums) - 1:
        return [letter for letter in letters[nums[index]]]
    list_ = cartesian(nums, letters, index+1)
    new_list = [[letter + s for s in list_] for letter in letters[nums[index]]]
    return [item for sublist in new_list for item in sublist]

### Minimum Window Substring

Bruteforce, O(n^2)

There is an O(n) algorithm.

1. Use two pointers: start and end to represent a window.
2. Move end to find a valid window.
3. When a valid window is found, move start to find a smaller window.

To check if a window is valid, we use a map to store (char, count) for chars in t. And use counter for the number of chars of t to be found in s. The key part is m[s[end]]--;. We decrease count for each char in s. If it does not exist in t, the count will be negative.

To really understand this algorithm, please see my code which is much clearer, because there is no code like if(map[s[end++]]++>0) counter++;.

    string minWindow(string s, string t) {
        unordered_map<char, int> m;
        // Statistic for count of char in t
        for (auto c : t) m[c]++;
        // counter represents the number of chars of t to be found in s.
        size_t start = 0, end = 0, counter = t.size(), minStart = 0, minLen = INT_MAX;
        size_t size = s.size();

        // Move end to find a valid window.
        while (end < size) {
            // If char in s exists in t, decrease counter
            if (m[s[end]] > 0)
                counter--;
            // Decrease m[s[end]]. If char does not exist in t, m[s[end]] will be negative.
            m[s[end]]--;
            end++;
            // When we found a valid window, move start to find smaller window.
            while (counter == 0) {
                if (end - start < minLen) {
                    minStart = start;
                    minLen = end - start;
                }
                m[s[start]]++;
                // When char exists in t, increase counter.
                if (m[s[start]] > 0)
                    counter++;
                start++;
            }
        }
        if (minLen != INT_MAX)
            return s.substr(minStart, minLen);
        return "";
    }

In [25]:
import collections
import sys

def minWindowSubstring(text, pattern):
    counter, count = collections.Counter(pattern), len(pattern)
    start = end = 0
    n = len(text)
    min_len = sys.maxint
    min_start = 0
    
    while end < n:
        if counter[text[end]] > 0:
            count -= 1
        counter[text[end]] -= 1
        
        while count == 0:
            if min_len > end - start + 1:
                min_len = end - start + 1
                min_start = start
            
            counter[text[start]] += 1
            if counter[text[start]] > 0:
                count += 1
            start += 1
        end += 1

    if min_len == sys.maxint:
        return ''
    else:
        return text[min_start:(min_start + min_len)]

assert minWindowSubstring('ADOBECODEBANC', 'ABC') == 'BANC'
r*g8ciyOpcLassert minWindowSubstring('AB', 'A') == 'A'
assert minWindowSubstring('AA', 'A') == 'A'
assert minWindowSubstring('AB', 'C') == ''
assert minWindowSubstring('ABA', 'AA') == 'ABA'

In [ ]:
### Contiguous sum to a target

### Balance parentheses

In [ ]:
def balance_parentheses(string):
    stack = []
    for c in string:
        if c == '(':
            stack.append(c)
        elif c == ')':
            if stack:
                stack.pop()
            else:
                return False
    return len(stack) == 0

assert balance_parentheses('(())') == True
assert balance_parentheses('()()') == True
assert balance_parentheses('(()') == False
assert balance_parentheses('()())') == False

### You have a string of numbers, i.e. 123. You can insert a + or - sign in front of ever number, or you can leave it empty. Find all of the different possibilities, make the calculation and return the sum. For example; +1+2+3 = 6 +12+3 = 15 +123 = 123 +1+23 = 24 ... -1-2-3 = 6 ... Return the sum of all the results.

In [17]:
def find_string_set(string, index, ops, possible_ops, string_set):
    if index == len(string) - 1:
        for op in possible_ops:
            ops[index] = op
            string_set.add(''.join([ops[i] + string[i] for i in range(len(string))]))
        return
    for op in possible_ops:
        ops[index] = op
        find_string_set(string, index + 1, ops, possible_ops, string_set)

string = '123'
ops = len(string)*['+']
possible_ops = set(['+', '-', ''])
string_set = set()
find_string_set(string, 0, ops, possible_ops, string_set)
sum_ = 0
for string in string_set:
    if string[0] == '+':
        string = '0' + string
    sum_ += eval(string)
print sum_

3

### Ways to score S
point denominators p = {3, 5, 10}
```
N = (S+1)*[0]
N[0] = 1
for s = 1 to S
  for i = 1 to n where n is the nummber of point denominators
    if s >= p[i]
      N[s] = N[s] + N[s-p[i]]
return N[S]
```

### Score combinations that sum to S
```
N = (S + 1)*[0]
N[0] = 1
for i = 1 to n
  for s = p[i] to S
    N[s] = N[s] + N[s-p[i]]
```

### Connected components in a graph
```
num_components = 0
for v in g.vertices:
  if discovered[v] = false
    num_components++
    component = bfs(g, v)
    print each node in component
```

### Convert a BST to a sorted circular doubly-linked list in-place
Recursion
```
tree.prev = max(tree.left)
tree.next = min(tree.right)
```

In [1]:
class Tree(object):
    def __init__(self, data):
        self.data = data
        self.parent = None
        self.left = None
        self.right = None
        self.prev = None
        self.next = None

    def __str__(self):
        return str(self.data)

def insert(tree, data, parent=None):
    if not tree:
        tree = Tree(data)
        tree.parent = parent
        if parent.data > data:
            parent.left = tree
        else:
            parent.right = tree
    else:
        if tree.data > data:
            insert(tree.left, data, tree)
        else:
            insert(tree.right, data, tree)

def inorder_print(tree):
    if tree:
        inorder_print(tree.left)
        print tree.data
        inorder_print(tree.right)
        
def find_min(tree):
    if not tree:
        return
    if not tree.left:
        return tree
    return find_min(tree.left)

def find_max(tree):
    if not tree:
        return
    if not tree.right:
        return tree
    return find_max(tree.right)
    
def convert(tree):
    if not tree:
        return
    if not tree.left and not tree.right:
        return
    tree.prev = find_max(tree.left)
    if tree.prev:
        tree.prev.next = tree
    tree.next = find_min(tree.right)
    if tree.next:
        tree.next.prev = tree
    convert(tree.left)
    convert(tree.right)

tree = Tree(4)
insert(tree, 2)
insert(tree, 1)
insert(tree, 3)
insert(tree, 5)

min_tree = find_min(tree)
assert min_tree.data == 1

max_tree = find_max(tree)
assert max_tree.data == 5

head = find_min(tree)
convert(tree)

list_ = head
while list_:
    print list_.data, list_.prev, list_.next
    list_ = list_.next

1 None 2
2 1 3
3 2 4
4 3 5
5 4 None


Insert a node to a list while doing reverse inorder traversal.

In [17]:
class Tree(object):
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

class Node(object):
    def __init__(self, data):
        self.data = data
        self.next = None
        self.prev = None
        
    def __str__(self):
        return str(self.data)
        
def reverse_inorder(tree, l):
    if not tree:
        return
    if tree.right:
        l = reverse_inorder(tree.right, l)
    l = insert_list(l, tree.data)
    if tree.left:
        l = reverse_inorder(tree.left, l)
    return l

def insert_list(l, data):
    new_head = Node(data)    
    new_head.next = l
    if l:
        l.prev = new_head
    return new_head

def print_list(l):
    while l:
        print l
        l = l.next
        
def reverse_print_list(tail):
    while tail:
        print tail
        tail = tail.prev

tree = Tree(4)
insert(tree, 2)
insert(tree, 1)
insert(tree, 3)
insert(tree, 5)

l = reverse_inorder(tree, None)
print_list(l)

1
2
3
4
5


### Ensure that there are a minimum of n dashes between any two of the same characters of a string
Be thorough considering all possible conditions including:
'-abc'
'a-b'
'abc--'
```
# i = 0
# while i < len(s)
#   if s[i] = s[i+1]
#     return False
#   if s[i] = '-'
#     k = 1
#     before = s[i-1]
#     j = i + 1
#     while j < len(s) and s[j] = '-'
#       k += 1
#       j += 1
#     after = s[j]
#     if before = after and k != n
#       return False
#     i = j
# return True
```

In [1]:
def dash(s, n):
    i = 0
    while i < len(s)-1:
        if s[i] == '-':
            k = 1
            before = 'b'
            if i > 0:
                before = s[i-1]
            j = i + 1
            while j < len(s) and s[j] == '-':
                k += 1
                j += 1
            if j < len(s):
                after = s[j]
            if before == after and not k == n:
                return False
            i = j
            continue
        else:
            if s[i] == s[i+1]:
                return False
            i += 1
    return True

assert dash('aa', 2) == False
assert dash('a-a', 2) == False
assert dash('a--a', 2) == True
assert dash('a-b', 2) == True

### rotate a matrix 90 degrees clockwise

O(n^2) for time, O(nm) for space

In [18]:
def rotate(matrix):
    if not type(matrix) == list:
        print 'Illegal argument'
        return
    m = len(matrix)
    n = len(matrix[0])
    new_matrix = [[0 for j in range(n)] for i in range(m)]
    for i in range(m):
        for j in range(n):
            new_matrix[j][m-i-1] = matrix[i][j]
    return new_matrix

assert rotate([[1, 2], [3, 4]]) == [[3, 1], [4, 2]]

If the matrix is square, O(1) for space is possible.

In [35]:
def swap_diagonal(m):
    side = len(m)
    for i in range(side):
        for j in range(side-i):
            swap(m, i, j, side-j-1, side-i-1)

def swap(m, i, j, k, l):
    m[i][j], m[k][l] = m[k][l], m[i][j]

def flip_vertical(m):
    side = len(m)
    for i in range(side):
        for j in range(side):
            swap(m, i, j, side-1-1, j)

m = [[1, 2], [3, 4]]
swap_diagonal(m)
flip_vertical(m)
print m

[[3, 1], [4, 2]]


### calculate the dot product of two sparse vectors
A sparse vector can be in a list of (index, value) tuples or a linked list of (index, value) tuples. A linear algorithm is similar to the merge step of merge sorting. O(n1 + n2)

In [15]:
# This algorithm assumes the input vectors (index, value) tuples are sorted by indices.

def dot_product(x, y):
    sum_ = 0
    i, j = 0, 0
    while i < len(x) and j < len(y):
        index1, value1 = x[i]
        index2, value2 = y[j]
        if index1 == index2:
            sum_ += value1*value2
            i += 1
            j += 1
        elif index1 > index2:
            j += 1
        else:
            i += 1
    return sum_

assert dot_product([(1, 1), (3, 3), (5, 5)], [(2, 2), (3, 3), (5, 5)]) == 34       

A hash table provides easier random access to elements, but an array performs better in accessing elements successively.

### Print tree levelwise
BFS

In [8]:
from collections import deque

class Tree(object):
    def __init__(self, data):
        self.data = data
        self.parent = None
        self.left = None
        self.right = None
        self.prev = None
        self.next = None

    def __str__(self):
        return str(self.data)

def insert(tree, data, parent=None):
    if not tree:
        tree = Tree(data)
        tree.parent = parent
        if parent.data > data:
            parent.left = tree
        else:
            parent.right = tree
    else:
        if tree.data > data:
            insert(tree.left, data, tree)
        else:
            insert(tree.right, data, tree)

In [28]:
def level_order_print(tree):
    queue = deque()
    queue.append(tree)
    while queue:
        count = len(queue)

        while count:
            tree = queue.popleft()
            print tree,
            for child in [node for node in (tree.left, tree.right) if node]:
                queue.append(child)
            count -= 1
        print

tree = Tree(4)
insert(tree, 2)
insert(tree, 1)
insert(tree, 3)
insert(tree, 5)
level_order_print(tree)

4
2 5
1 3


### Number of paths from one element of a 2D array to another
Dynamic programming
```
N[0, 0] = 1
N[i, 0] = 1
N[0, j] = 1
N[i, j] = N[i-1, j] + N[i, j-1]
```

### Return the head node of the singly linked list with each pair of nodes swapped. If there is a last odd node leave it in place. Example: Input: 1 -> 2 -> 3 -> 4 -> 5 Output: 2 -> 1 -> 4 -> 3 -> 5

Swap values recursively rather than nodes.

In [1]:
def swap_pairs(l):
    if l and l.next:
        l.data, l.next.data = l.next.data, l.data
        swap_pairs(l.next.next)  
    
class Node(object):
    def __init__(self, x):
        self.data = x
        self.next = None
    
    def __str__(self):
        return str(self.data)

def print_list(l):
    while l:
        print l
        l = l.next

head = Node(1)
head.next = Node(2)
head.next.next = Node(3)
head.next.next.next = Node(4)
head.next.next.next.next = Node(5)

swap_pairs(head)
print_list(head)

2
1
4
3
5


### It was a mathematical questions about finding the greatest number (but less than the given number) by reordering the digits in the number.
Here is an algorithm for the next higher number.
```
Compare the two digits from the tail and move up until digits[i] < digits[i-1]. Call this position i a pivot.
Among the digits right to the pivot, find the digit whose value is closest to the pivot value.
Swap the two.
Sort the digits right to the pivot.
```

### Given a list of chars, return the 2nd most frequently occurring char
    Go through the list.
    Store the count of each char in a hash table.
    Go through the hash table.
    Return the second largest count.
    
Instead of sorting the counts, go through the k, v pairs. Be careful keeping track of the second largest count because there are multiple first largest counts.

    if count > first_max
        second_max = first_max
        first_max = count
    elif count != first_max and count > second_max
        second_max = count
   
O(n) for time and O(1) for space

### Given various time intervals throughout the day, return time(s) when no time interval is scheduled
    Sort the intervals by their start times.
    Go through the intervals.
    Increment count as an interval starts.
    Decrement count as an interval ends.
    When count becomes zero, append the time to result.
    Return the result after going through all intervals.
    
The implementation below is similar to the merge step in mergesort. O(n) for both time and space. Be careful not to forget the case where a meeting starts immediately as another ends.

In [5]:
from collections import deque

def rooms_required(times):
    start = deque([i for i, j in times])
    end = deque([j for i, j in times])
    count = 0
    max_count = 0
    while start and end:
        if start[0] < end[0]:
            start.popleft()
            count += 1
            max_count = max(max_count, count)
        elif start[0] > end[0]:
            end.popleft()
            count -= 1
        else:
            start.popleft()
            end.popleft()

    while end:
        end.popleft()
        count -= 1
    return max_count

schedule = [(1, 3), (2, 4), (5, 6)]

assert rooms_required(schedule) == 2

def free_time(times):
    start = deque([i for i, j in times])
    end = deque([j for i, j in times])
    count = 0
    max_count = 0
    isFree = False
    while start and end:
        if start[0] < end[0]:
            start_time = start.popleft()
            count += 1
            if isFree and count == 1:
                isFree = False
                print 'Free time ends at', start_time
            max_count = max(max_count, count)
        elif start[0] > end[0]:
            end_time = end.popleft()
            count -= 1
            if count == 0:
                isFree = True
                print 'Free time starts at', end_time
        else:
            start.popleft()
            end.popleft()

    while end:
        end.popleft()
        count -= 1

free_time(schedule)

Free time starts at 4
Free time ends at 5


### Permutations of a string
    perms = permutations(word[1:])
    for perm in perms
        insert word[0] on each position of perm including head and tail
    return all words afer insertions

### Given an array of integers find a contiguous subset that sums to a given number
    Start from the first element.
    Add one element at a time until sum gets larger than or equals to the target.
    If the sum equals to the target, return start and end
    else substract a[start] from the sum and increment start
    Repeat adding elements.
    
This works for an array of positive integers. O(n)

In [10]:
def contiguous_sum(array, target):
    sum_ = 0
    start = 0
    for end in range(len(array)):
        sum_ += array[end]
        while sum_ > target:
            sum_ -= array[start]
            start += 1
        if sum_ == target:
            return array[start:end+1]
    
assert contiguous_sum([15, 2, 4, 8, 9, 5, 10, 23], 23) == [2, 4, 8, 9]
assert contiguous_sum([23, 2, 4, 8, 9, 5, 10, 23], 23) == [23]
assert contiguous_sum([24, 2, 4, 8, 9, 5, 10, 23], 23) == [2, 4, 8, 9]

### Reverse a linked list

In [19]:
class Node(object):
    def __init__(self, data):
        self.data = data
        self.next = None
        
    def __str__(self):
        return str(self.data)
        
def traverse_list(list_):
    while list_:
        print list_
        list_ = list_.next

def reverse(list_):
    prev = None
    while list_:
        next_ = list_.next
        list_.next = prev
        prev = list_
        list_ = next_
    head = prev
    return head

head = Node(1)
head.next = Node(2)
head.next.next = Node(3)

head = reverse(head)
traverse_list(head)

3
2
1


Use recursion.

In [20]:
def reverse(l):
    head = l
    head = reverse_util(l, None)
    return head

def reverse_util(l, prev):
    if not l:
        return prev

    next_ = l.next
    l.next = prev
    return reverse_util(next_, l)

head = Node(1)
head.next = Node(2)
head.next.next = Node(3)

head = reverse(head)
traverse_list(head)

3
2
1


### Square each element of a sorted array of a mixture of positive and negative integers and keep the array sorted
    Partition the positive and negative values.
    Square each element.
    Merge the two partitions but the negative array should be in the reverse order from the tail to the head.

### Columnwise print of a binary tree
    Find min and max horizontal distances from the root.
    For col = min_dist, max_dist
        Do bfs and print if hd equals to col.

In [20]:
import sys

class Tree(object):
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        self.parent = None
        
    def __str__(self):
        return str(self.data)

root = Tree(1)
root.left = Tree(2)
root.right = Tree(3)
root.left.left = Tree(4)
root.left.right = Tree(5)
root.right.left = Tree(6)

min_dist = sys.maxint
max_dist = -sys.maxint-1

def columnwise_print(tree):
    find_minmax_dist(tree, 0)
    for col in range(min_dist, max_dist+1):
        hdistance(tree, col, 0)
        print

def find_minmax_dist(tree, hd):
    global min_dist, max_dist
    if not tree:
        return
    if hd < min_dist:
        min_dist = hd
    if hd > max_dist:
        max_dist = hd
    find_minmax_dist(tree.left, hd-1)
    find_minmax_dist(tree.right, hd+1)
    
def hdistance(tree, col, hd):
    if not tree:
        return
    if col == hd:
        print tree,
    hdistance(tree.left, col, hd-1)
    hdistance(tree.right, col, hd+1)
    
columnwise_print(root)

4
2
1 5 6
3


In [25]:
def find_range(tree, min_dist, max_dist):
    if not tree.left and not tree.right:
        return min_dist, max_dist
    if tree.left:
        min_dist, _ = find_range(tree.left, min_dist-1, max_dist)
    if tree.right:
        _, max_dist = find_range(tree.right, min_dist, max_dist+1)
    return min_dist, max_dist

find_range(root, 0, 0)

(-2, 1)

More efficient algorithm. Put tree values in a hash table while measuring distances. The keys of the table are the horizontal distance of each node from the root. Unlike the implementation above, this one takes one traversal. O(nlogn) for sorting the distances.

In [38]:
from collections import defaultdict

columns = defaultdict(lambda: [])

class Solution(object):
    def verticalOrder(self, root):
        """
        :type root: TreeNode
        :rtype: List[List[int]]
        """
        hdist(root, 0)
        sorted_dist = sorted(columns.keys())
#         result = (sorted_dist[-1] - sorted_dist[0] + 1)*[0]
        result = []
        for i, dist in enumerate(sorted_dist):
#             result[i] = columns[dist]
            result.append(columns[dist])
            
        return result

def hdist(tree, dist):
    if not tree:
        return
    columns[dist].append(tree.val)
    hdist(tree.left, dist - 1)
    hdist(tree.right, dist + 1)
    
s = '[3,9,8,4,0,1,7]'
codec = Codec()
tree = codec.deserialize(s)
sol = Solution()
sol.verticalOrder(tree)

NameError: name 'Codec' is not defined

In [ ]:
from collections import defaultdict

columns = defaultdict(lambda: [])

class Solution(object):
    def verticalOrder(self, root):
        """
        :type root: TreeNode
        :rtype: List[List[int]]
        """
        hdist(root, 0)
        sorted_dist = sorted(columns.keys())
        result = (sorted_dist[-1] - sorted_dist[0] + 1)*[0]
        for i, dist in enumerate(sorted_dist):
            result[i] = columns[dist]
            
        return result

def hdist(tree, dist):
    if not tree:
        return
    hdist(tree.left, dist - 1)
    print tree.val
    hdist(tree.right, dist + 1)
    
s = '[3,9,8,4,0,1,7]'
codec = Codec()
tree = codec.deserialize(s)
sol = Solution()
sol.verticalOrder(tree)

### Power of Two
0 = 00, 2 = 10, 4 = 100, 8 = 1000, ...

n & (n-1) eliminates the least significant 1.

In [1]:
def powerOfTwo(n):
    return n & (n-1) == 0

assert powerOfTwo(7) == False
assert powerOfTwo(8) == True

### Number of 1's in the binary representation of an integer
n & (n-1) eliminates the least significant 1. Count how many & operations are needed to make the integer zero.

In [4]:
def numberOfOnes(x):
    count = 0
    while x:
        x = x & (x-1)
        count += 1
    return count

assert numberOfOnes(3) == 2

### Check full binary tree
Check from the top to the bottom because in this way we can terminate early as soon as we find out that the tree is not full.

A tree is full only if the tree has both left and children and both left and right subtrees are full.

In [4]:
class Tree(object):
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        self.parent = None
        
    def __str__(self):
        return str(self.data)

def check_full_tree(tree):
    if not tree:
        return True
    if (not tree.left and tree.right) or (tree.left and not tree.right):
        return False
    return check_full_tree(tree.left) and check_full_tree(tree.right)
    
root = Tree(1)
root.left = Tree(2)
root.right = Tree(3)
root.left.left = Tree(4)
root.left.right = Tree(5)
root.right.left = Tree(6)

assert check_full_tree(root) == False

root.right.right = Tree(7)
assert check_full_tree(root) == True

### Check if a tree is complete
A tree is complete when all levels are full except for the bottom level. The  bottom level doesn't have to be full but should be filled up from the left. Picture a binary heap. If you index from the top, the index of the left child of a tree is *2i+1* while that of the right child is *2i+2*. The index of each tree should strictly less than the number of nodes.

`max_index = n_nodes - 1`

In [4]:
class Tree(object):
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        self.parent = None
        
    def __str__(self):
        return str(self.data)
    
max_index = 0
node_count = 0
def check_complete_tree(tree):
    def complete_tree_util(tree, index):
        global max_index, node_count
        if not tree:
            return
        node_count += 1
        max_index = max(max_index, index)
        complete_tree_util(tree.left, 2*index+1)
        complete_tree_util(tree.right, 2*index+2)
    
    complete_tree_util(tree, 0)
    return max_index == node_count - 1
    

root = Tree(1)
root.left = Tree(2)
root.right = Tree(3)
root.left.left = Tree(4)
root.left.right = Tree(5)    

assert check_complete_tree(root) == True

root = Tree(1)
root.left = Tree(2)
root.right = Tree(3)
root.left.left = Tree(4)
root.left.right = Tree(5)
root.right.right = Tree(6)

assert check_complete_tree(root) == False

### Check if two strings are anagrams
A. Sort, O(nlogn)

    Sort the two strings
    Compare the two sorted stings
    
B. Count, three passes of O(n)

    Count the letters in the first string
    Count the letters in the second string
    Compare the counts

### given a list of words, group anagrams
A. Hashing, O(n*hash)

    For each word
        Sort the word and calculate hash. The hash value should be same for the anagrams.
        Put the word in the hash table whose keys are the hash values and the values are lists of words.

B. Sort, O(nmlogm + nlogn) where m is the avg. length of strings and n is the length of the list.

    Create an array that stores the index of each string.
    Sort each string.
    Now the list is grouped by anagrams.
    If a group consists of one string, it doesn't have an anagram in the list.

### Check if a graph is bipartite
    for v in g.vertices
        color[v] = BLACK
        bfs(g, v)
        
    function process_edge(u, v)
        if color[u] = color[v]
            graph is not bipartite
        color[v] = complement(color[u])

### Write a function that finds the square root of a decimal number.
r^2 = n

Binary search between 1 <= r <= n

terminal conditions:
- mid*mid == n
- high - low == 1

In [ ]:
def binary_search(x, low, high):
    if high - low == 1:
        return low
    if low < high:
        mid = (low + high)//2
        mid_square = mid*mid
        if mid_square == x:
            return mid
        if mid_square > x:
            return binary_search(x, low, mid)
        else:
            return binary_search(x, mid, high)

binary_search(n, 1, n)

### Find k-th largest element in a tree
    Initiate an empty min heap.
    Do bfs starting from the root.
    At each node, if the item is greater than the min of the heap, pop the min and insert the item to the heap.
    In the end return the min of the heap.

In [8]:
import heapq
from collections import deque

class Tree(object):
    def __init__(self, data):
        self.data = data
        self.children = []
        
def kth_largest(tree, k):
    heap = []
    queue = deque()
    
    queue.append(tree)
    while queue:
        tree = queue.popleft()
        for child in tree.children:
            queue.append(child)
            if len(heap) < k:
                heapq.heappush(heap, child.data)
            else:
                if child.data < heapq.nsmallest(1, heap):
                    heapq.heappushpop(heap, child.data)
    return heapq.nsmallest(1, heap)

tree1 = Tree(1)
tree2 = Tree(2)
tree3 = Tree(3)
tree4 = Tree(4)
tree5 = Tree(5)
tree1.children = [tree2, tree3]
tree2.children = [tree4, tree5]

assert kth_largest(tree1, 4)[0] == 2

### Find k-th largest element in a binary search tree
    Do reverse in-order traversal.
    Count each node.
    When count == k, print the node.

In [3]:
class Tree(object):
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        self.parent = None
        
    def __str__(self):
        return str(self.data)

rank = 0
def kth_largest(tree, k):
    global rank
    if tree:
        right = kth_largest(tree.right, k)
        if right:
            return right
        rank += 1
        if rank == k:
            return tree
        left = kth_largest(tree.left, k)
        if left:
            return left

root = Tree(4)
root.left = Tree(2)
root.right = Tree(5)
root.left.left = Tree(1)
root.left.right = Tree(3)
root.right.right = Tree(6)

assert kth_largest(root, 3).data == 4

### How can one implement a queue with only a stack implementation?
    function enqueue(x)
        while outstack
            instack.push(outstack.pop())
        instack.push(x)
    
    function dequeue()
        while instack
            outstack.push(instack.pop())
        return outstack.pop()

### Print the Fibonacci sequence to the Nth digit.
    back2 = 0
    back1 = 1
    fib = back1 + back2
    while len(str(fib)) < n
        print fib
        back2 = back1
        back1 = fib
        fib = back1 + back2

### Determine the 10 most frequent words given a terabyte of strings
    Tokenize the strings.
    Count the appearances of the tokens.
    As going through the (token, count) tuples, maintain a min heap of size 10.

### Given some email ids, and a similarity function which says whether two email ids are similar, determine all the sets of email ids that are similar to each other.

Clarify that if similar(a, b) and similar(b, c) means similar(a, c). If similarity(a, b) = TRUE, there is an edge between a and b. The nodes in a connected component are similar to each other.

### Clone a linked list with next and random pointer
    Copy each node and insert between the original node and the next node.
    For each node
        l.next.random = l.random.next
    Detach the copied nodes from original nodes and return the copied head.
    
O(n) time and O(1) space

In [1]:
class Node(object):
    def __init__(self, data):
        self.data = data
        self.next = None
        self.random = None

def print_list(l):
    if l:
        print l.data
        print_list(l.next)
    
def insert_nodes(l):
    if l:
        next_ = l.next
        l.next = Node(l.data)
        l.next.next = next_
        insert_nodes(l.next.next)
        
def set_random(l):
    if not l:
        return
    if l.next:
        l.next.random = l.random.next
        set_random(l.next.next)

def detach(l):
    if not l:
        return
    if l.next:
        next_ = l.next
        l.next = l.next.next
        detach(next_)
    
head = Node(1)
head.next = Node(2)
head.next.next = Node(3)

head.random = head.next.next
head.next.random = head
head.next.next.random = head.next

insert_nodes(head)
set_random(head)
clone_head = head.next
detach(head)

print_list(head)
print_list(clone_head)

1
2
3
1
2
3


### Longest palindromic substring
The center of a palindrome can be either one or two letters.
    
    For each letter in the string
        Expand both left and right from the current letter as the center
        Do the same with the current letter and the next one as the center 
        Keep track of the max length

O(n^2) time and O(1) space

In [11]:
def palindrome_util(s, start, end):
    n = len(s)
    while start >= 0 and end < n and s[start] == s[end]:
        start -= 1
        end += 1
    start += 1
    end -= 1
    return start, end
    
assert palindrome_util('dabae', 2, 2) == (1, 3)
assert palindrome_util('dabbae', 2, 3) == (1, 4)

def check_length(start, end, max_len, max_start):
    length = end - start + 1
    if max_len < length:
        max_len = length
        max_start = start
    return max_len, max_start
    
def palindrome(s):
    max_len = 0
    max_start = 0
    for i in range(len(s)-1):
        start, end = palindrome_util(s, i, i)
        max_len, max_start = check_length(start, end, max_len, max_start)
        start, end = palindrome_util(s, i, i + 1)
        max_len, max_start = check_length(start, end, max_len, max_start)
    return s[max_start:max_start + max_len]

assert palindrome('abacdeffed') == 'deffed'
assert palindrome('abc') == 'a'

### Sort a nearly sorted array each element of which is at most k steps away from its sorted position
A. Insertion sort. For each element a[i], search a[0] ~ a[i-1] for the right position. The search takes actually at most k times. O(nk)

B. Create a heap of k elements. Pot the min and append to the result. Insert an element from the rest and put into the heap. O(k) for building heap. O((n-k)logk) for insertion to the heap.

In [15]:
def insertion_sort(a):
    for i in range(1, len(a)):
        j = i
        while j >= 0 and a[j-1] > a[j]:
            swap(a, j-1, j)
            j -= 1

def swap(a, i, j):
    a[i], a[j] = a[j], a[i]

a = [2, 6, 3, 12, 56, 8]
insertion_sort(a)
assert a == [2, 3, 6, 8, 12, 56]

### Sort a nearly sorted array in which two elements are out of order
1. Search for an out-of-order element from the left.
2. Search for an out-of-order element from the right.
3. Swap the two.

O(n)

In [2]:
def one_swap_sort(a):
    n = len(a)
    i = 1
    while i < n and a[i] < a[i+1]:
        i += 1
    low = i
    
    j = n - 1
    while j > i and a[j-1] < a[j]:
        j -= 1
    high = j
    
    a[low], a[high] = a[high], a[low]
    
a = [2, 6, 3, 12]
one_swap_sort(a)
assert a == [2, 3, 6, 12]

### Find the indices of the max value in a large array
Linear search, O(n)

In [3]:
import sys

def max_indices(a):
    max_value = -sys.maxint - 1
    for i, v in enumerate(a):
        if v > max_value:
            max_value = v
            max_indices = [i]
        elif v == max_value:
            max_indices.append(i)

    return max_indices

assert max_indices([1, 2, 3, 2, 3]) == [2, 4]

### Quicksort

In [11]:
def quicksort(a, low, high):
    if low < high:
        pivot = partition(a, low, high)
        quicksort(a, low, pivot-1)
        quicksort(a, pivot+1, high)

def partition(a, low, high):
    pivot = high
    first_high = low
    for i in range(low, high):
        if a[i] < a[pivot]:
            swap(a, i, first_high)
            first_high += 1
    swap(a, pivot, first_high)
    return first_high

def swap(a, i, j):
    a[i], a[j] = a[j], a[i]

a = [2, 6, 3, 12, 56, 8]
quicksort(a, 0, len(a)-1)
assert a == [2, 3, 6, 8, 12, 56]

### Mergesort

In [7]:
from collections import deque

def mergesort(a, low, high):
    if low < high:
        mid = (low + high)//2
        mergesort(a, low, mid)
        mergesort(a, mid+1, high)
        merge(a, low, mid, high)
        
def merge(a, low, mid, high):
    low_queue = deque(a[low:mid+1])
    high_queue = deque(a[mid+1:high+1])
    i = low
    while low_queue and high_queue:
        if low_queue[0] < high_queue[0]:
            a[i] = low_queue.popleft()
        else:
            a[i] = high_queue.popleft()
        i += 1
    
    while low_queue:
        a[i] = low_queue.popleft()
        i += 1
        
    while high_queue:
        a[i] = high_queue.popleft()
        i += 1
        
a = [2, 6, 3, 12, 56, 8]
mergesort(a, 0, len(a)-1)
assert a == [2, 3, 6, 8, 12, 56]

### Implement queue using a circular array
Maintain the head and next pointers. The pointers can wrap around to the beginning of the array. The index must be (*i % size*). Resize the array as the number of items  increase beyond or decrease below a certain number.

In [20]:
class Queue(object):
    def __init__(self):
        self.size = 10
        self.data = self.size*[0]
        self.count = 0
        self.head = 0
        self.next = 0
        
    def enqueue(self, data):
        if self.count == self.size:
            self.resize(self.size*2)
        try:
            self.data[self.next] = data
        except IndexError:
            print 'Invalid the next pointer'
        self.count += 1
        self.next = (self.next + 1) % self.size
    
    def dequeue(self):
        data = self.data[self.head]
        self.head = (self.head + 1) % self.size
        self.count -= 1
        if self.count < self.size//4:
            self.resize(self.size//2)
        return data
    
    def resize(self, new_size):
        tmp = new_size*[0]
        i = 0
        while i < self.count:
            index = (i + self.head) % self.size
            try:
                tmp[i] = self.data[index]
            except IndexError:
                print 'Invalid index while resizing '
            i += 1
        self.head = 0
        self.next = i
        self.data = tmp
        self.size = new_size
        
queue = Queue()
queue.enqueue(1)
queue.enqueue(2)
assert queue.dequeue() == 1
assert queue.dequeue() == 2

### K smallest number in an integer array

- max binary heap: O(nlogk)
- quickselection, i.e, partition in quicksort: O(n) expected, O(n^2) worst case
- median of medians: O(n) worst case

In [ ]:
# Quickselection
# k-th largest is the k-th element in the sorted order

import random

def kth_largest(a, low, high, k):
    pivot = random.randint(low, high)
    swap(a, pivot, high)
    pivot = partition(a)
    rank = pivot - low + 1
    if rank < k:
        kth_largest(a, pivot+1, high, k-rank)
    elif rank > k:
        kth_largest(a, low, pivot-1, k)
    else:
        return a[pivot]
        
def swap(a, i, j):
    a[i], a[j] = a[j], a[i]

def partition(a, low, high, pivot):
    first_high = 0
    for i, x in enumerate(a):
        if x < a[high]:
            swap(a, i, first_high)
            first_high += 1
    swap(a, first_high, high)
    return first_high

In [5]:
# There is a bug in this code. If n is not a multiple of 5, it may break, either the index error or a worng answer.

def kth_smallest(a, left, right, k):
    length = right - left + 1
    if  length <= 5:
        return sorted(a[left:right+1])[k-1]

    medians = [kth_smallest(a, left+5*i, left+5*(i+1)-1, 3) for i in range(length//5)]  
    pivot_value = kth_smallest(medians, 0, len(medians)-1, len(medians)//2+1)
    pivot = partition(a, left, right, pivot_value)

    rank = pivot - left + 1
    if rank < k:
        return kth_smallest(a, pivot+1, right, k - rank)
    elif rank > k:
        return kth_smallest(a, left, pivot-1, k)
    else:
        return a[pivot]
    
def partition(a, left, right, pivot_value):
    first_high = left
    for i in range(left, right+1):
        if a[i] < pivot_value:
            a[i], a[first_high] = a[first_high], a[i]
            first_high += 1
    return first_high

a = [2, 9, 3, 4, 1, 7, 8, 5, 6, 10, 11]
assert kth_smallest(a, 0, len(a)-1, 1) == 1
assert kth_smallest(a, 0, len(a)-1, 2) == 2
assert kth_smallest(a, 0, len(a)-1, 3) == 3
assert kth_smallest(a, 0, len(a)-1, 4) == 4
assert kth_smallest(a, 0, len(a)-1, 5) == 5
assert kth_smallest(a, 0, len(a)-1, 6) == 6
assert kth_smallest(a, 0, len(a)-1, 7) == 7
assert kth_smallest(a, 0, len(a)-1, 8) == 8
assert kth_smallest(a, 0, len(a)-1, 9) == 9

AssertionError: 

If there are duplicates in the array, the algorithms above will find k-th smallest in the sorted order. If k is small, a set can be used instead of heap.

In [ ]:
n = len(nums)

cache = set(nums[:3])
for k in nums[3:]:
    if len(cache) < 3:
        cache.add(k)
        continue
    if k not in cache and k > min(cache):
        cache.remove(min(cache))
        cache.add(k)
return min(cache) if len(cache) == 3 else max(cache)

In [ ]:
Given a set of inputs <number, userid> in a log file: log: <number, userid> example: 1,2 1,1 2,1 3,1 1,2 out: <number, count> 1,2 2,1 3,1 The output should be all the unique numbers and the count associated with them.

In [ ]:
There is two dimensional array where each sub array (row) is sorted, i.e. [[1 1000 2000] [20 10001 5000] [55 10002 222222]] Find a minimum range contain a number from each row. For above array it should be (1000-1002) range.

### Serialize and deserialize a BST
Serialization of a BST is just to store keys in pre-order.

Deserialization is similar to check_bst.

O(n)

In [2]:
class Tree(object):
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        self.parent = None

    def __str__(self):
        return str(self.data)

def preorder_print(tree):
    if tree:
        print tree,
        preorder_print(tree.left)
        preorder_print(tree.right)

In [4]:
import sys

def deserialize(a, lb, ub):
    global index
    if index >= len(a):
        return
    tree = None
    key = a[index]
    if lb <= key <= ub: # The equal signs will allow duplicates in the tree.
        tree = Tree(key)
        index += 1
        tree.left = deserialize(a, lb, key)
        tree.right= deserialize(a, key, ub)
    return tree

global index
a = [10, 5, 1, 7, 50, 50]
index = 0
lb = -sys.maxint - 1
ub = sys.maxint
tree = deserialize(a, lb, ub)
preorder_print(tree)

10 5 1 7 50 50


In [5]:
def serialize(tree, data):
    if tree:
        data.append(tree.data)
        serialize(tree.left, data)
        serialize(tree.right, data)

data = []
serialize(tree, data)
assert data == a

### Serialize and deserialize a complete binary tree
Imagine a binary heap. O(n)

In [20]:
def deserialize(data, index):
    if not data:
        return
    if index >= len(data):
        return
    tree = Tree(data[index])
    tree.left = deserialize(data, 2*index+1)
    tree.right = deserialize(data, 2*index+2)
    return tree

data = [1, 2, 5, 3, 4]
tree = deserialize(data, 0)
preorder_print(tree)

1 2 3 4 5


In [22]:
def serialize(tree, data, index):
    if tree:
        data[index] = tree.data
        serialize(tree.left, data, 2*index+1)
        serialize(tree.right, data, 2*index+2)

num_nodes = 5
data = num_nodes*[None]
serialize(tree, data, 0)
assert data == [1, 2, 5, 3, 4]

The recursive serialization above requires the number of nodes in the tree. The iterative implementation below doesn't but uses some extra memory for a queue. It is basically the level-order print of a binary tree.

In [24]:
from collections import deque

def serialize(tree, data):
    if not tree:
        return
    queue = deque()
    queue.append(tree)
    while queue:
        count = len(queue)
        while count:
            tree = queue.popleft()
            data.append(tree.data)
            if tree.left:
                queue.append(tree.left)
            if tree.right:
                queue.append(tree.right)
            count -= 1

data = []
serialize(tree, data)
assert data == [1, 2, 5, 3, 4]

### Serialize and deserialize a binary tree
Fill the null node with a special character, e.g., '/'. O(n) time

In [28]:
def deserialize(data):
    global index
    if index >= len(data):
        return
    key = data[index]
    index += 1
    if key == '/':
        return
    tree = Tree(key)
    tree.left = deserialize(data)
    tree.right = deserialize(data)
    return tree

data = 'ABD//EG///C/F//' # I think I can remove the trailing slashes without making it ambiguous.
index = 0
tree = deserialize(data)
preorder_print(tree)

A B D E G C F


In [32]:
def serialize_util(tree, data):
    if tree:
        data.append(tree.data)
        serialize_util(tree.left, data)
        serialize_util(tree.right, data)
    else:
        data.append('/')

def serialize(tree, data):
    data = []
    serialize_util(tree, data)
    return ''.join(data)

assert serialize(tree, data) == 'ABD//EG///C/F//'

### Serialize and deserialize an n-ary tree
Mark the end of a branch with ). See https://www.cs.usfca.edu/~brooks/S04classes/cs245/lectures/lecture11.pdf.

This tree `'ABE)FK)))C)DG)H)I)J)))'` can be read as below:

```ABE,End of E, FK, End of K, End of F, C, End of C, DG, End of G, ..., J, End of J, End of D, End of A```

Serialization: DFS recursively (DFS is basically the pre-order traversal. In process_vertex_eary(), append the input tree. In process_vertex_late(), mark the tree processed.

Deserialization: Pre-order traversal.

In [51]:
class Tree(object):
    def __init__(self, data):
        self.data = data
        self.children = []

    def __str__(self):
        return str(self.data)

def preorder_print(tree):
    if tree:
        print tree,
        for child in tree.children:
            preorder_print(child)

def deserialize(data):
    global index
    if index >= len(data):
        return
    if data[index] == ')':
        index += 1
        return
    tree = Tree(data[index])
    index += 1
    child = deserialize(data)
    while child:
        tree.children.append(child)
        child = deserialize(data)
    return tree

data = 'ABE)FK)))C)DG)H)I)J)))'
index = 0
tree = deserialize(data)
preorder_print(tree)

A B E F K C D G H I J


In [55]:
def serialize_util(tree, data):
    if tree:
        data.append(tree.data)
        for child in tree.children:
            serialize_util(child, data)
        data.append(')')
    
def serialize(tree):
    data = []
    serialize_util(tree, data)
    return ''.join(data)
    
assert serialize(tree) == 'ABE)FK)))C)DG)H)I)J)))'

### Swap k-th and (n-k)-th nodes in a linked list
Clarify if it's allowed to swap just data in those nodes. Otherwise, have to manipulate their pointers.

Consider all possible cases:
- k < n - k
- k > n - k
- k = n - k
- k and (n-k) are right next to each other.
- Either of k or (n-k) is eithr head or tail.

### A 2D matrix is sorted horizontally and vetically. Find an element.
Find the row that contains the element using a variant of binary search. The terminating condition is when

```m[mid][0] <= x <= m[mid][-1]```

rather than

```m[mid] == x```

in the 1D array case.

FInd the column that contains the element in the row using binary search.

In [2]:
def binary_search(a, x, low, high):
    if low > high:
        return None
    mid = (low + high)//2
    if a[mid] == x:
        return mid
    if a[mid] > x:
        return binary_search(a, x, low, mid-1)
    else:
        return binary_search(a, x, mid+1, high)
    
a = [1, 2, 3, 5, 8, 9]
x = 3
n = len(a)
assert binary_search(a, x, 0, n-1) == 2

In [19]:
def find_row(a, x, low, high):
    if low > high:
        return None
    mid = (low + high)//2
    if a[mid][0] <= x <= a[mid][-1]:
        return mid
    if a[mid][0] > x:
        return find_row(a, x, low, mid-1)
    else:
        return find_row(a, x, mid+1, high)
    
a = [[1, 2, 3], [5, 8, 9], [12, 15, 20]]
x = 15
m = len(a)
assert find_row(a, x, 0, m-1) == 2

In [21]:
def find_col(a, x, low, high, row):
    if low > high:
        return None
    mid = (low + high)//2
    if a[row][mid] == x:
        return mid
    if a[row][mid] > x:
        return find_col(a, x, low, mid-1, row)
    else:
        return find_col(a, x, mid+1, high, row)
    
a = [[1, 2, 3], [5, 8, 9], [12, 15, 20]]
x = 15
n = len(a[0])
row = 2
assert find_col(a, x, 0, n-1, row) == 1

In [25]:
def search_2dmatrix(a, x):
    m = len(a)
    n = len(a[0])
    row = find_row(a, x, 0, m-1)
    col = find_col(a, x, 0, n-1, row)
    return row, col

a = [[1, 2, 3], [5, 8, 9], [12, 15, 20]]
x = 15
assert search_2dmatrix(a, x) == (2, 1)

### Among a list of points in the 2D coordinate system, find the maximum number of points on a line

Among the points on a line, (y2 - y1)/(x2 - x1) is constant. Keep the counts in a hash table. The key should not be slopes due to the precision issue, e.g., 1/4 may not be equal to 2/8 in floating-point numbers. Use the tuple of the nominator and the denominator after reducing the pair with gcd, i.e.,

```((x2-x1), (y2-y1))```


### Given a linked list, remove the nth node from the end of list and return its head.

For example,

   Given linked list: 1->2->3->4->5, and n = 2.

   After removing the second node from the end, the linked list becomes 1->2->3->5.
Note:
Given n will always be valid.
Try to do this in one pass.

As you traverse the list, store each element to a hash table. The keys are the distance from the head.

### Build a maximum binary tree
Given an integer array with no duplicates. A maximum tree building on this array is defined as follow:

The root is the maximum number in the array.

The left subtree is the maximum tree constructed from left part subarray divided by the maximum number.

The right subtree is the maximum tree constructed from right part subarray divided by the maximum number.

In [14]:
class Tree(object):
    def __init__(self, data):
        self.left = None
        self.right = None
        self.data = data
        
    def __str__(self):
        return str(self.data)

def build_max_binary_tree(a, low, high):
    if low <= high:
        max_value = max(a[low:high+1])
        max_index = a.index(max_value)
        tree = Tree(max_value)
        tree.left = build_max_binary_tree(a, low, max_index-1)
        tree.right = build_max_binary_tree(a, max_index+1, high)
        return tree
    
a = [3,2,1,6,0,5]
root = build_max_binary_tree(a, 0, len(a)-1)
level_order_print(root)

6
3 5
2 0
1


### Find leaves of binary trees

In [19]:
def find_leaves(tree, leaves):
    if not tree:
        return
    if not tree.left and not tree.right:
        leaves.append(tree.data)
    find_leaves(tree.left, leaves)
    find_leaves(tree.right, leaves)

leaves = []
find_leaves(root, leaves)
assert sorted(leaves) == [0, 1]

### Find duplicates from an array of integers. The integers can be between 1 and len(array) inclusively.
The range is the hint. Each element can be use as an index.

In [21]:
def find_duplicaes(a):
    for x in a:
        i = abs(x) - 1
        if a[i] < 0:
            print abs(x)
        else:
            a[i] *= -1
        
a = [4,3,2,7,8,2,3,1]
find_duplicaes(a)

2
3


### 513. Find Bottom Left Tree Value
BFS but add children to the queue from the right to the left. The bottom left node is the one that is added to the queue the last.

In [29]:
def bottom_left_node(tree):
    queue = []
    queue.append(tree)
    while queue:
        tree = queue.pop(0)
        for child in [node for node in (tree.right, tree.left) if node]:
            queue.append(child)
    return child

print bottom_left_node(root)

1


### 515. Find Largest Value in Each Tree Row
Similar to levelwise printing of a tree

In [30]:
def level_max(tree):
    queue = [tree]
    while queue:
        max_value = max([node.data for node in queue])
        print max_value
        count = len(queue)
        while count:
            tree = queue.pop(0)
            for child in [node for node in (tree.left, tree.right) if node]:
                queue.append(child)
            count -= 1
level_max(root)

6
5
2
1


### Sums of subtrees

In [36]:
from collections import defaultdict

def subtree_sum(tree, counts):
    if not tree:
        return 0
    sum_ = tree.data + subtree_sum(tree.left, counts) + subtree_sum(tree.right, counts)
    counts[sum_] += 1
    return sum_

counts = defaultdict(lambda: 0)
tree = Tree(5)
tree.left = Tree(2)
tree.right = Tree(-5)
subtree_sum(tree, counts)
print counts

counts = defaultdict(lambda: 0)
tree = Tree(5)
tree.left = Tree(2)
tree.right = Tree(-3)
subtree_sum(tree, counts)
print counts

defaultdict(<function <lambda> at 0x7ff2f874b938>, {2: 2, -5: 1})
defaultdict(<function <lambda> at 0x7ff2f8616668>, {2: 1, 4: 1, -3: 1})


### Number of islands in 2D grid

bfs variant

In [8]:
from collections import defaultdict

discovered = defaultdict(lambda: False)

class Solution(object):
    def numIslands(self, grid):
        """
        :type grid: List[List[str]]
        :rtype: int
        """
        if not grid:
            return 0
        n_rows = len(grid)
        n_cols = len(grid[0])
        count = 0
        for i in range(n_rows):
            for j in range(n_cols):
                if not discovered[(i, j)] and int(grid[i][j]):
                    bfs(grid, (i, j))
                    count += 1
        return count

def bfs(grid, start):
    print start
    discovered[start] = True
    queue = [start]
    while queue:
        u = queue.pop(0)
        i, j = u
        for v in [(x, y) for (x, y) in [(i-1, j), (i+1, j), (i, j-1), (i, j+1)] if x >=0 and x < len(grid) and y >= 0 and y < len(grid[0])]:
            iv, jv = v
            if not discovered[v] and int(grid[iv][jv]):
                discovered[v] = True
                queue.append(v)
                
solution = Solution()
# grid = ["11000","11000","00100","00011"]
# solution.numIslands(grid)
grid = ["11110","11010","11000","00000"]
solution.numIslands(grid)

(0, 0)


1

### Two sum in a binary tree

In [29]:
class Solution(object):
    def findTarget(self, root, k):
        """
        :type root: TreeNode
        :type k: int
        :rtype: bool
        """
        cache = set()
        def findTargetUtil(tree, k):
            if not tree:
                return False
            if tree.val in cache:
                return True
            complement = k - tree.val
            if not complement == tree.val:
                cache.add(complement)
            if findTargetUtil(tree.left, k):
                return True
            if findTargetUtil(tree.right, k):
                return True
            return False
        
        return findTargetUtil(root, k)

def search(tree, x):
    if not tree:
        return False
    if tree.val == x:
        return True
    if tree.val > x:
        return search(tree.left, x)
    else:
        return search(tree.right, x)

class Tree(object):
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None

tree = Tree(2)
tree.left = Tree(1)
tree.right = Tree(3)

assert search(tree, 1) == True
assert search(tree, 2) == True
assert search(tree, 3) == True
assert search(tree, 4) == False

sol = Solution()
sol.findTarget(tree, 4)

True

### Find the first bad version in a list of versions 1...n.

Binary search. But be careful that output is not the index of index but the actual version number, i.e., index + 1.

In [38]:
cache = {}
def binary_search(low, high):
    if low < high:
        mid = (low + high) // 2 + 1
        if not mid in cache:
            cache[mid] = isBadVersion(mid)
        if not mid-1 in cache:
            cache[mid-1] = isBadVersion(mid-1)
        if not cache[mid-1] and cache[mid]:
            return mid
        if cache[mid]:
            return binary_search(low, mid-1)
        else:
            return binary_search(mid+1, high)
    else:
        return low

badVersions = [False, True, True]
def isBadVersion(k):
    return badVersions[k-1]
binary_search(1, 2)

2

In [42]:
class SparseMatrix(object):
    def __init__(self, m):
        self.data = {}
        if not m:
            return
        if type(m) == list:
            for i in range(len(m)):
                if type(m[0]) == list:
                    for j in range(len(m[0])):
                        if m[i][j]:
                            self.data[(i, j)] = m[i][j]
                else:
                    self.data[(i, 0)] = m[i]
        else:
            self.data[(0, 0)] = m

def init_matrix(n_rows, n_cols):
    return [[0 for j in range(n_cols)] for i in range(n_rows)]

class Solution(object):
    def multiply(self, A, B):
        """
        :type A: List[List[int]]
        :type B: List[List[int]]
        :rtype: List[List[int]]
        """
        # (1, 1) = (1, :)(:, 1)
        # (1, 2) = (1, :)(:, 2)
        
        # Special cases
        # The input matrices are vectors.
        
        n_rows_A = len(A)
        n_cols_A = len(A[0])
        n_rows_B = len(B)
        n_cols_B = len(B[0])
        A = SparseMatrix(A)
        B = SparseMatrix(B)
        C = SparseMatrix(None)
        n_rows = n_rows_A
        n_cols = n_cols_B
        for i in sorted([i for (i, _)  in A.data.keys()]):
            for j in sorted([j for (_, j) in B.data.keys()]):
                sum_ = 0
                for j_inner in range(n_cols_A):
                    if (i, j_inner) in A.data and (j_inner, j) in B.data:
                        sum_ += A.data[(i, j_inner)]*B.data[(j_inner, j)]
                C.data[(i, j)] = sum_
        result = init_matrix(n_rows, n_cols)        
        for (i, j), v in C.data.iteritems():
            result[i][j] = v
        return result

A = [[1,0,0],[-1,0,3]]
B = [[7,0,0],[0,0,0],[0,0,1]]
sol = Solution()
assert sol.multiply(A, B) == [[7, 0, 0], [-7, 0, 3]]

In [10]:
class Solution(object):
    def diameterOfBinaryTree(self, root):
        """
        :type root: TreeNode
        :rtype: int
        """
        # graph with single node
        # linear graph
        if not root:
            return 0
        self.max_diameter = 0
        def height(tree):
            if not tree:
                return -1

            h1 = max(0, height(tree.left))
            h2 = max(0, height(tree.right))
            n_children = h1 > 0 + h2 > 0
            self.max_diameter =  max(self.max_diameter, h1 + h2 + n_children)
            return max(h1, h2) + 1
        height(root)
        return self.max_diameter

class Tree(object):
    left = None
    right = None

tree = Tree()
tree.left = Tree()

sol = Solution()
sol.diameterOfBinaryTree(tree)

1

### Check if a linked list is a palindrome

Find a median. While doing so revers the first half. Progress the reversed first half and the second half to compare each node. While doing so reverse the first half back to the original order.

In [29]:
class Solution(object):
    def isPalindrome(self, head):
        """
        :type head: ListNode
        :rtype: bool
        """
        # single node
        # two nodes
        # odd no. of nodes
        # even no. nodes
        
        # Reverse a linked list
        #
        # l = head
        # while l:
        #     next_ = l.next
        #     l.next = prev
        #     prev = l
        #     l = next_

        # a
        # a a
        # a b a
        # a b c
        # a b b a
        # a b b c
        
        if not head:
            return True

        if not head.next:
            return True

        isPalindrome = True
        slow = head
        fast = head
        prev = None
        while slow and fast and fast.next:
            fast = fast.next.next
            next_ = slow.next
            slow.next = prev
            prev = slow
            slow = next_
        
        l1 = prev
        prev = slow
        if fast: # Odd no of nodes
            l2 = slow.next
        else:
            l2 = slow
        while l1 and l2:
            if not l1.val == l2.val:
                isPalindrome = False
            next_ = l1.next
            l1.next = prev
            prev = l1
            l1 = l1.next
            l2 = l2.next
            
        return isPalindrome
    
class ListNode(object):
    def __init__(self, x):
        self.val = x
        self.next = None
        
l = ListNode(1)
l.next = ListNode(2)
l.next.next = ListNode(2)
l.next.next = ListNode(1)
sol = Solution()
sol.isPalindrome(l)

True

### Level-order serialization

Similar to level-order printing.

No trailing null's necessary.

The deserialization logic should be same as the serialization.

In [1]:
# Definition for a binary tree node.
class TreeNode(object):
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

class Codec:

    def serialize(self, root):
        """Encodes a tree to a single string.
        
        :type root: TreeNode
        :rtype: str
        """
        result = []
        queue = [root]
        while queue:
            tree = queue.pop(0)
            if tree:
                result.append(str(tree.val))
                for child in [tree.left, tree.right]:
                    queue.append(child)
            else:
                result.append('null')
        for i, item in enumerate(reversed(result)):
            if not item == 'null':
                break
        trailing_null_start = -i
        if trailing_null_start:
            result = result[:trailing_null_start]
        result = '[' + ','.join(result) + ']'
        return result
        

    def deserialize(self, data):
        """Decodes your encoded data to tree.
        
        :type data: str
        :rtype: TreeNode
        """
        data = data[1:-1].split(',')
        return deserialize_util(data, 0)

        
def deserialize_util(data, index):
    if not data:
        return
    val = data.pop(0)
    if val == 'null':
        return
    root = TreeNode(int(val))
    queue = [root]
    while data:
        tree = queue.pop(0)
        if data:
            val = data.pop(0)
            if not val == 'null':
                tree.left = TreeNode(int(val))
                queue.append(tree.left)
        if data:
            val = data.pop(0)
            if not val == 'null':
                tree.right = TreeNode(int(val))
                queue.append(tree.right)
    return root
        
# Your Codec object will be instantiated and called as such:

codec = Codec()
tree5 = TreeNode(5)
tree2 = TreeNode(2)
tree3 = TreeNode(3)
tree4 = TreeNode(4)
tree5.left = tree2
tree5.right = tree3
tree22 = TreeNode(2)
tree3.left = tree22
tree44 = TreeNode(4)
tree3.right = tree44
tree33 = TreeNode(3)
tree22.left = tree33
tree22.right = TreeNode(1)

tree = codec.deserialize(codec.serialize(tree5))
codec.serialize(tree)

'[5,2,3,null,null,2,4,3,1]'

### 238. Product of Array Except Self
The alogorithm below won't scale.

In [1]:
class Solution(object):
    def productExceptSelf(self, nums):
        """
        :type nums: List[int]
        :rtype: List[int]
        """
        
        # 1 0 2
        # 0 1 2
        # 0 0 2
        # 1 -1
        # -1 -1
        # 1 -1 -1
        
        zero_count = 0
        prod = 1
        n = len(nums)
        for i, x in enumerate(nums):
            if x == 0:
                zero_count += 1
                zero_index = i
                continue
            prod *= x
        
        if zero_count == 1:
            result =n*[0]
            result[zero_index] = prod
            return result
        elif zero_count > 1:
            return n*[0]

        result = []
        for x in nums:
            count = 0
            sum_ = 0
            while not sum_ == abs(prod):
                sum_ += abs(x)
                count += 1
            result.append(-count if (prod < 0 and x > 0) or (prod > 0 and x < 0) else count)
        return result
    
nums = [5,9,2,-9,-9,-7,-8,7,-9,10]
sol = Solution()
sol.productExceptSelf(nums)

[-51438240,
 -28576800,
 -128595600,
 28576800,
 28576800,
 36741600,
 32148900,
 -36741600,
 28576800,
 -25719120]

In [8]:
class Solution(object):
    def productExceptSelf(self, nums):
        """
        :type nums: List[int]
        :rtype: List[int]
        """
        
        # 1 0 2
        # 0 1 2
        # 0 0 2
        # 1 -1
        # -1 -1
        # 1 -1 -1
        
    
        n = len(nums)
        result = n*[0]
        tmp = 1
        i = 0
        while i < n:
            result[i] = tmp
            tmp *= nums[i]
            print result
            i += 1
        print result

        i = n - 1
        tmp = 1
        while i >= 0:
            result[i] *= tmp
            tmp *= nums[i]
            i -= 1

        return result
    
# nums = [5,9,2,-9,-9,-7,-8,7,-9,10]
nums = [1, 0]
sol = Solution()
sol.productExceptSelf(nums)

[1, 0]
[1, 1]
[1, 1]


[0, 1]

### 127. Word Ladder

BFS. The words with the distance of one are neighbors. Note that the end word is added to the word list in the beginning and that the number of levels to the endWord is counted.

In [12]:
class Solution(object):
    def ladderLength(self, beginWord, endWord, wordList):
        """
        :type beginWord: str
        :type endWord: str
        :type wordList: List[str]
        :rtype: int
        """
        if not wordList:
            return 0
        queue = [beginWord]
        result = [beginWord]
        dist = 0
        if endWord not in wordList:
            wordList.append(endWord)
        while queue:
            count = len(queue)
            u = queue.pop(0)
            indices = []
            while count:
                for i, word in enumerate(wordList):
                    if distanceOne(u, word):
                        if word == endWord:
                            result.append(endWord)
                            return dist + 1
                        queue.append(word)
                        indices.append(i)
                count -= 1
            dist += 1
            wordList = [word for i, word in enumerate(wordList) if i not in indices]
        return 0

from collections import Counter            

def distanceOne(w1, w2):
    count = 0
    for i, _ in enumerate(w1):
        if not w1[i] == w2[i]:
            count += 1
    return count <= 1

wordList = ["hot","dot","dog","lot","log","cog"]
sol = Solution()
sol.ladderLength('hit', 'cog', wordList)

5

### Sparse Matrix Multiplication

In [ ]:
class SparseMatrix(object):
    def __init__(self, m):
        self.data = {}
        if not m:
            return
        if type(m) == list:
            for i in range(len(m)):
                if type(m[0]) == list:
                    for j in range(len(m[0])):
                        if m[i][j]:
                            self.data[(i, j)] = m[i][j]
                else:
                    self.data[(i, 0)] = m[i]
        else:
            self.data[(0, 0)] = m

def init_matrix(n_rows, n_cols):
    return [[0 for j in range(n_cols)] for i in range(n_rows)]

class Solution(object):
    def multiply(self, A, B):
        """
        :type A: List[List[int]]
        :type B: List[List[int]]
        :rtype: List[List[int]]
        """
        # (1, 1) = (1, :)(:, 1)
        # (1, 2) = (1, :)(:, 2)
        
        # Special cases
        # The input matrices are vectors.
        
        n_rows_A = len(A)
        n_cols_A = len(A[0])
        n_rows_B = len(B)
        n_cols_B = len(B[0])
        A = SparseMatrix(A)
        B = SparseMatrix(B)
        C = SparseMatrix(None)
        n_rows = n_rows_A
        n_cols = n_cols_B
        for i in sorted([i for (i, _)  in A.data.keys()]):
            for j in sorted([j for (_, j) in B.data.keys()]):
                sum_ = 0
                for j_inner in range(n_cols_A):
                    if (i, j_inner) in A.data and (j_inner, j) in B.data:
                        sum_ += A.data[(i, j_inner)]*B.data[(j_inner, j)]
                C.data[(i, j)] = sum_
        result = init_matrix(n_rows, n_cols)        
        for (i, j), v in C.data.iteritems():
            result[i][j] = v
        return result

In [14]:
# A sparse matrix as a sequance of (index, value)
# Assume that the sequance is sorted by indices.

# max_col can be incorrect if there are trailing zero columns.
def transpose(sm):
    max_col = 0
    for row in sm:
        for col, _ in row:
            max_col = max(max_col, col)

    smt = [[] for i in range(max_col+1)]
    for i, row in enumerate(sm):
        for col, v in row:
            smt[col].append((i, v))
    return smt

def dot_product(x, y):
    sum_ = 0
    i, j = 0, 0
    while i < len(x) and j < len(y):
        index1, value1 = x[i]
        index2, value2 = y[j]
        if index1 == index2:
            sum_ += value1*value2
            i += 1
            j += 1
        elif index1 > index2:
            j += 1
        else:
            i += 1
    return sum_

def multiply(sm1, sm2):
    new_matrix = []
    sm2t = transpose(sm2)
    for i, row in enumerate(sm1):
        new_row = []
        if row:
            for j, col in enumerate(sm2t):
                product = dot_product(row, col)
                if product:
                    new_row.append((j, product))
        new_matrix.append(new_row)
    return new_matrix

sm = [[(0, 1), (1, 2)],[],[(0, 3), (1, 4)]]
smt = transpose(sm)
print sm
print smt
                               
multiply(sm, smt)

[[(0, 1), (1, 2)], [], [(0, 3), (1, 4)]]
[[(0, 1), (2, 3)], [(0, 2), (2, 4)]]


[[(0, 5), (2, 11)], [], [(0, 11), (2, 25)]]

###  Longest pair chain

You are given n pairs of numbers. In every pair, the first number is always smaller than the second number.

Now, we define a pair (c, d) can follow another pair (a, b) if and only if b < c. Chain of pairs can be formed in this fashion.

Given a set of pairs, find the length longest chain which can be formed. You needn't use up all the given pairs. You can select pairs in any order.

This is a variation of the longest increasing subsequence problem. Note that the pairs has to be sorted first because, unlike the longest increasing subsequence, pairs can be selected in any order. Sort maybe by the frist element. Not quite sure.

In [12]:
class Solution(object):
    def findLongestChain(self, pairs):
        """
        :type pairs: List[List[int]]
        :rtype: int
        """
        n = len(pairs)
        l = n*[1]
        pairs = sorted(pairs, key=lambda (x, y): x)
        for i in range(1, n):
            max_length = 0
            for j in range(i):
                if pairs[i][0] > pairs[j][1]:
                    max_length = max(max_length, l[j])
            l[i] = max_length + 1
        return max(l)

pairs = [[1,2], [2,3], [3,4]]
sol = Solution()
sol.findLongestChain(pairs)

2

### Longest palindrome subsequence

Does this work?

In [26]:
class Solution(object):
    def longestPalindromeSubseq(self, s):
        """
        :type s: str
        :rtype: int
        """
        n = len(s)
        if n == 1:
            return 1
        l = [[0 for i in range(n)] for j in range(n)]
        for i in range(n):
            l[i][i] = 1
        for j in range(n):
            for i in range(j):
                if s[i] == s[j]:
                    l[i][j] = l[i+1][j-1] + 2
                else:
                    l[i][j] = max(l[i+1][j], l[i][j-1])
                    
#         print l
        return max([l[i][j] for i in range(n) for j in range(n)])
    
s = "cbbd"
s = "euazbipzncptldueeuechubrcourfpftcebikrxhybkymimgvldiwqvkszfycvqyvtiwfckexmowcxztkfyzqovbtmzpxojfofbvwnncajvrvdbvjhcrameamcfmcoxryjukhpljwszknhiypvyskmsujkuggpztltpgoczafmfelahqwjbhxtjmebnymdyxoeodqmvkxittxjnlltmoobsgzdfhismogqfpfhvqnxeuosjqqalvwhsidgiavcatjjgeztrjuoixxxoznklcxolgpuktirmduxdywwlbikaqkqajzbsjvdgjcnbtfksqhquiwnwflkldgdrqrnwmshdpykicozfowmumzeuznolmgjlltypyufpzjpuvucmesnnrwppheizkapovoloneaxpfinaontwtdqsdvzmqlgkdxlbeguackbdkftzbnynmcejtwudocemcfnuzbttcoew"
sol = Solution()
sol.longestPalindromeSubseq(s)

49

### Sliding window maximum

Given an array nums, there is a sliding window of size k which is moving from the very left of the array to the very right. You can only see the k numbers in the window. Each time the sliding window moves right by one position.

In [34]:
from collections import deque
class Solution(object):
    def maxSlidingWindow(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: List[int]
        """
        dq = deque()
        n = len(nums)
        result = []
        for i in range(n):
            # Clean up the leftover from the last window
            while dq and dq[0] < (i - k + 1):
                dq.popleft()
            # Have the current max in the window be at the head of dq.
            while dq and nums[i] > nums[dq[-1]]:
                dq.pop()
            dq.append(i)
            # Once reach to the end of the current window, collect the max in the window.
            if i - k + 1 >= 0:
                result.append(nums[dq[0]])
        return result

[]

### Best time to buy and sell a stock

In [7]:
import sys

def stock_trade(prices):
    min_price = sys.maxint
    min_price_index = -1
    max_profit = -sys.maxint - 1
    max_price_index = -1
    for i, price in enumerate(prices):
        if price < min_price:
            min_price = price
            min_price_index = i
        profit = price - min_price
        if profit > max_profit:
            max_profit = profit
            max_profit_index = i
    
    if max_profit == 0: # if the trend is monotonically downward
        return 0, 0
    else:
        return min_price_index, max_profit_index

prices = [7, 1, 5, 3, 6, 4]
assert stock_trade(prices) == (1, 4)

prices = [7, 6, 4, 3, 1]
assert stock_trade(prices) == (0, 0)

In [20]:
from collections import defaultdict

def word_pattern(s, i, p, j, table):
    if i == len(s) and j == len(p):
        return True
    elif i == len(s) or j == len(p):
        return False
    
    if table[p[j]]:
        if s[i:i+len(table[p[j]])] == table[p[j]]:
            return word_pattern(s, i+len(table[p[j]]), p, j+1, table)
        else:
            return False
    else:
        for k in range(i, len(s)):
            table[p[j]] = s[i:k+1]
            if word_pattern(s, k+1, p, j+1, table):
                return True
            table[p[j]] = None # For backtracking
        return False

s = "redblueredblue"
p = 'abab'
table = defaultdict(lambda: None)
assert word_pattern(s, 0, p, 0, table) == True

p = "aaaa"
s = "asdasdasdasd"
table = defaultdict(lambda: None)
assert word_pattern(s, 0, p, 0, table) == True

p = "aabb"
s = "xyzabcxzyabc"
table = defaultdict(lambda: None)
assert word_pattern(s, 0, p, 0, table) == False

reusable

In [3]:
def backtrack(result, temp_list, nums, remain, start):
    if remain < 0:
        return
    elif remain == 0:
        result.append(list(temp_list))
    else:
        i = start
        while i < len(nums) and nums[i] <= target:
            temp_list.append(nums[i])
            backtrack(result, temp_list, nums, remain - nums[i], i)
            temp_list.pop()
            i += 1
            
def combination_sum(nums, target):
    nums = sorted(nums)
    result = []
    temp_list = []
    backtrack(result, temp_list, nums, target, 0)
    return result

nums = [2, 6, 3, 7]
target = 7
combination_sum(nums, target)

[[2, 2, 3], [7]]

not reusable

In [8]:
def backtrack(result, temp_list, nums, remain, start):
    if remain < 0:
        return
    elif remain == 0:
        result.append(list(temp_list))
    else:
        i = start
        while i < len(nums) and nums[i] <= target:
            if not nums[i] == nums[i-1]:
                temp_list.append(nums[i])
                backtrack(result, temp_list, nums, remain - nums[i], i+1)
                temp_list.pop()
            i += 1
            
def combination_sum(nums, target):
    nums = sorted(nums)
    result = []
    temp_list = []
    backtrack(result, temp_list, nums, target, 0)
    return result

nums = [2, 6, 3, 2, 7]
target = 7
combination_sum(nums, target)

[[6]]

In [11]:
def factor_combinations(result, combi, n):
    i = 2 if not combi else combi[-1]
    while i <= n//i:
        if n%i == 0:
            combi.append(i)
            combi.append(n/i)
            result.append(list(combi))
            combi.pop()
            factor_combinations(result, combi, n//i)
            combi.pop()
        i += 1

result = []
factor_combinations(result, [], 12)
print result

[[2, 6], [2, 2, 3], [3, 4]]


In [ ]:
def sudoku(board):
    n_rows = len(board)
    n_cols = len(board[0])
    used_rows = [[]]
    used_cols = {}
    used_inner = {}
    for i in range(n_rows):
        for j in range(n_cols):
            if not board[i][j] == '.':
                num = int(board[i][j])
                k = i/3*3 + j/3
                if used_rows[i][num] or user_cols[j][num] or used_inner[k][num]:
                    return False
                used_rows[i][num] = used_cols[j][num] = used_inner[k][num] = True
    return True

In [54]:
def word_break(s, words):
    n = len(s)
    dp = n*[0]
    for i in range(n):
        for w in words:
            if s[i-len(w)+1:i+1] == w and (i-len(w)+1 == 0 or dp[i-len(w)]):
                dp[i] = True
                break
    return dp[-1]

words = ['leet', 'code']
assert word_break('leetcode', words) == True
assert word_break('leetcod', words) == False